# Introduction

## Python

pip install [package] : command line to install package for python e.g.

!pip install numpy

install numpy for python in jupyter notebook(exclamation mark is used to run command line in jupyter notebook)

import [package] : import package for python

import numpy as np : import numpy for python and use np as alias

## Numpy

numpy : package for python to do numerical computation(running C code behind the scene to speed up the computation)

np.array([1,2,3]) : create array in numpy, `list <-> np.array`

## Pandas

pandas : package for python to do data analysis(based on numpy). You can think it as excel in python. Often import as pd(e.g. import pandas as pd)

pd.DataFrame({'a':[1,2,3],'b':[4,5,6]}) : create dataframe in pandas, dictionary <-> pd.DataFrame, `excel table <-> pd.DataFrame`

## Jupyter Notebook and Google Colab

Jupyter : browser application to run python code and add description in the same file

[filename].ipynb : jupyter notebook file

Colab : jupyter notebook in the Google Cloud resources so you don't need to install jupyter notebook in your computer

kernel(runtime) : python code running in the background

cell : a block of code in jupyter notebook. You can run the code in a cell by pressing `shift+enter`

## JSON

JSON : JavaScript Object Notation. A format to store data in a file(e.g. .json file). This is a standard format to store data in a file across different programming languages.

In JSON, data is stored in key-value pairs or arrays. The key is a string and the value can be a string, number, boolean, null, array or object. Mostly `dictionary <-> JSON` in this file.

## Workflow

| sg_num | k_label | pg_name | irrep_name |
|--------|---------|----------|------------|

(sg_num)  ----- get_momenta_table ---->  (k_label, vector)

(sg_num, k_label) ----- get_irreps_table, convert_bilbao_format ----> (pg_name, irrep_name, matrix)

| momenta_table | sg_irreps_table |
|---------------|-----------------|

(sg_irreps_table) ----- direct_square ----> (character_array) ----- get_decomposition ----> (pg_irreps_arr)

| character_arr | pg_irreps_arr |
|---------------|---------------|


(pg_irreps_arr) ----- get_irreps_table ----> (pg_irreps_table)

In [52]:
trim_dict = {
    'monoclinic': {
        11: ['C', 'D', 'E', 'Z'],
        14: ['D', 'Z']
    },
    
    'orthorhombic_p': {
        51: ['R', 'S', 'U', 'X'],
        52: ['R', 'S', 'T', 'Y'],
        53: ['R', 'T', 'U', 'Z'],
        54: ['R', 'S', 'U', 'X'],
        55: ['R', 'S', 'T', 'U', 'X', 'Y'],
        56: ['R', 'S', 'T', 'U', 'X', 'Y'],
        57: ['R', 'S', 'T', 'U', 'Y', 'Z'],
        58: ['R', 'S', 'T', 'U', 'X', 'Y'],
        59: ['R', 'S', 'T', 'U', 'X', 'Y'],
        60: ['R', 'S', 'T', 'U', 'X', 'Z'],
        61: ['R', 'S', 'T', 'U', 'X', 'Y', 'Z'],
        62: ['R', 'S', 'T', 'U', 'X', 'Y', 'Z'],
    },
    
    'orthorhombic_c': {
        63: ['R', 'T', 'Z'],
        64: ['R', 'S', 'T', 'Z']
    },
    
    'tetragonal': {
        127: ['R', 'X', 'A', 'M'],
        128: ['R', 'X', 'A', 'M'],
        129: ['R', 'X', 'A', 'M'],
        130: ['R', 'X', 'A', 'M'],
        135: ['R', 'X', 'A', 'M'],
        136: ['R', 'X', 'A', 'M'],
        137: ['R', 'X', 'A', 'M'],
        138: ['R', 'X', 'A', 'M']
    },
    
    'hexagonal': {
        176: ['L', 'H', 'A'],
        193: ['L', 'A'], 
        194: ['L', 'H', 'A']
    },
    
    'cubic': {205: ['X', 'M']}
 }

# Import packages and set constants

In [100]:
import urllib
from urllib.request import urlopen
import numpy as np
import pandas as pd
import json
import os


In [ ]:
if(not os.path.exists('data.json')):
    url = 'https://raw.githubusercontent.com/ulissigroup/kptheory/master/data.json'
    urllib.request.urlretrieve(url, 'data.json')

with open('data.json') as f:
    data = json.load(f)
print(json.dumps(data, indent=2)[:1000])

# TRIM points

### Scraping Billbao server (cryst.ehu.es) 

In [35]:
def get_html_file(url, opts, verbose=False):
    """
    Fetches html file from url.
    
    Args:
        url (str): URL to fetch.
        opts (dict): Dictionary of options to pass to url.
        verbose (bool, optional): Set to 1 to print url, 0 to not print url. Defaults to 0.
    
    Returns:
        str: HTML file as a string.
    """
    url_address = url + '?' + urllib.parse.urlencode(opts)
    if verbose:
        print(url_address)
    url_stream = urlopen(url_address)
    html_file = url_stream.read().decode("utf8")
    url_stream.close()
    
    return html_file


In [114]:
def get_irreps_table(sg_num, k_label):
    """
    Fetches irreps table from cryst.ehu.es.
    
    Args:
        sg_num (int): Number of the space group.
        k_label (str): Vector point label.
    
    Returns:
        pd.DataFrame: Pandas DataFrame with irreps table.
    """
    url = 'https://www.cryst.ehu.es/cgi-bin/cryst/programs/representations_out.pl'
    opts = {
        'tipogrupo': '',
        'super': sg_num,
        'texto': '',
        'phys': '',
        'vecfinal': k_label,
        'list': 'Submit'
    }
    html_file = get_html_file(url, opts)
    df_list = pd.read_html(html_file)
    df = df_list[1][1:]
    df.columns = df_list[1].iloc[0]
    df.insert(0, 'Refined Symbol', df['Seitz Symbol'].apply(lambda x: x.split('|')[0][1:]), True)
    return df

# Example usage

#rawdf=get_irreps_table(52,'Y&(0,1/2,0)')
#raw_df=get_irreps_table(129,'M')
#rawdf=get_irreps_table(54,'C')
#rawdf=get_irreps_table(129,'A')
#rawdf=get_irreps_table(176,'H')
rawdf=get_irreps_table(205,'R')
#rawdf=get_irreps_table(193,'H')
rawdf.head()

,Refined Symbol,Matrix presentation,Seitz Symbol,R1+R3+,R1-R3-,R2+R2+,R2-R2-
1,1,1 0 0 t1 0 1 0 t2 0 0 1 t3,"{1|t1,t2,t3}",Cos(π(t1+t2+t3)) 0 Sin(π(t1+t2+t3)) 0 0 Cos(π(...,Cos(π(t1+t2+t3)) 0 Sin(π(t1+t2+t3)) 0 0 Cos(π(...,Cos(π(t1+t2+t3)) 0 Sin(π(t1+t2+t3)) 0 0 Cos(π(...,Cos(π(t1+t2+t3)) 0 Sin(π(t1+t2+t3)) 0 0 Cos(π(...
2,2001,-1 0 0 1/2 0 -1 0 0 0 0 1 1/2,"{2001|1/2,0,1/2}",0 0 0 1 0 0 1 0 0 -1 0 0 -1 0 0 0,0 0 0 1 0 0 1 0 0 -1 0 0 -1 0 0 0,0 0 0 1 0 0 1 0 0 -1 0 0 -1 0 0 0,0 0 0 1 0 0 1 0 0 -1 0 0 -1 0 0 0
3,2010,-1 0 0 0 0 1 0 1/2 0 0 -1 1/2,"{2010|0,1/2,1/2}",0 -1 0 0 1 0 0 0 0 0 0 -1 0 0 1 0,0 -1 0 0 1 0 0 0 0 0 0 -1 0 0 1 0,0 -1 0 0 1 0 0 0 0 0 0 -1 0 0 1 0,0 -1 0 0 1 0 0 0 0 0 0 -1 0 0 1 0
4,2100,1 0 0 1/2 0 -1 0 1/2 0 0 -1 0,"{2100|1/2,1/2,0}",0 0 1 0 0 0 0 -1 -1 0 0 0 0 1 0 0,0 0 1 0 0 0 0 -1 -1 0 0 0 0 1 0 0,0 0 1 0 0 0 0 -1 -1 0 0 0 0 1 0 0,0 0 1 0 0 0 0 -1 -1 0 0 0 0 1 0 0
5,3+111,0 0 1 0 1 0 0 0 0 1 0 0,"{3+111|0,0,0}",(1+√3)/4 (1+√3)/4 (√3-1)/4 (√3-1)/4 (√3-1)/4 (...,(1+√3)/4 (1+√3)/4 (√3-1)/4 (√3-1)/4 (√3-1)/4 (...,-1/2 -1/2 1/2 1/2 1/2 -1/2 1/2 -1/2 -1/2 -1/2 ...,-1/2 -1/2 1/2 1/2 1/2 -1/2 1/2 -1/2 -1/2 -1/2 ...


In [37]:
def get_momenta_table(sg_num):

    """
    Fetches momenta table from cryst.ehu.es.
    
    Args:
        sg_num (int): Number of the group.
    
    Returns:
        pd.DataFrame: Pandas DataFrame with momenta table.
    """

    url = 'https://www.cryst.ehu.es/cgi-bin/cryst/programs/representations_vec.pl'
    opts = {
        'tipogrupo': '',
        'super': sg_num,
        'complex': 'Submit'
    }
    html_file = get_html_file(url, opts)
    df_list = pd.read_html(html_file)
    df = df_list[5][1:]
    return df

# Example usage
rawdf2=get_momenta_table(193)
rawdf2

,0,1,2
1,NaN,L,"(1/2,0,1/2)"
2,NaN,M,"(1/2,0,0)"
3,NaN,H,"(1/3,1/3,1/2)"
4,NaN,K,"(1/3,1/3,0)"
5,NaN,A,"(0,0,1/2)"
6,NaN,GM,"(0,0,0)"
7,NaN,U,"(1/2,0,w)"
8,NaN,R,"(u,0,1/2)"
9,NaN,SM,"(u,0,0)"
10,NaN,LD,"(u,u,0)"


### Parsing Momenta Table

In [39]:
def mirror_direction(sg_num):
    """
    Fetches mirror direction from cryst.ehu.es.

    Args:
        sg_num (int): Number of the group.

    Returns:
        tuple: A tuple with (mirror_x, mirror_y, mirror_z), 
               where each element is a boolean value indicating 
               whether there is a mirror in that direction.
    """

    rawdf=get_irreps_table(sg_num,'GM')
    #find Refined Symbol column 2001
    mirror_z = rawdf[rawdf['Refined Symbol'].str.contains('2001')]['Seitz Symbol'].values[0].split('|')[1][:-1].split(',')[2]=='1/2'
    mirror_y = rawdf[rawdf['Refined Symbol'].str.contains('2010')]['Seitz Symbol'].values[0].split('|')[1][:-1].split(',')[1]=='1/2'
    mirror_x = rawdf[rawdf['Refined Symbol'].str.contains('2100')]['Seitz Symbol'].values[0].split('|')[1][:-1].split(',')[0]=='1/2'

    return mirror_x,mirror_y,mirror_z

def trim_k_labels(sg_num):
    """
    Trims k labels by removing labels corresponding to mirror directions.

    Args:
        sg_num (int): Number of the group.

    Returns:
        list: A list of TRIM k labels.
    """
    
    mirror_x,mirror_y,mirror_z=mirror_direction(sg_num)

    df_momenta = get_momenta_table(sg_num)

    df_noline= df_momenta[~df_momenta[2].str.contains('u') & ~df_momenta[2].str.contains('v') & ~df_momenta[2].str.contains('w')]

    df_klist = df_noline[2].apply(lambda x: x.replace('(','').replace(')','').split(','))
    
    return list(df_noline[df_klist.apply(lambda x: (mirror_x and x[0]!="0") | (mirror_y and x[1]!="0") | (mirror_z and x[2]!="0"))][1])

trim_k_labels(127)

['R', 'X', 'A', 'M']

In [40]:
def get_momenta_dict(sg_num):
    df_momenta = get_momenta_table(sg_num)
    df_noline= df_momenta[~df_momenta[2].str.contains('u') & ~df_momenta[2].str.contains('v') & ~df_momenta[2].str.contains('w')]
    df_klist = df_noline[2].apply(lambda x: x.replace('(','').replace(')','').split(','))
    return dict(zip(df_noline[1],df_klist))

get_momenta_dict(127)

{'R': ['0', '1/2', '1/2'],
 'X': ['0', '1/2', '0'],
 'A': ['1/2', '1/2', '1/2'],
 'GM': ['0', '0', '0'],
 'M': ['1/2', '1/2', '0'],
 'Z': ['0', '0', '1/2']}

## Get TRIM Dictionary

In [61]:
# Dictionary of TRIM k labels for each space group
trim_dict = {} # trim_dict[crystal_system][sg_num] = TRIM_k_labels list

# Monoclinic crystal system
trim_dict['monoclinic'] = {
    11: ['C','D','E','Z'],
    14: ['D','Z'],
}

# Orthorhombic Primitive crystal system
orthorhombicPlist = [ 51 , 52 , 53, 54, 55, 56, 57, 58, 59, 60, 61, 62]
trim_dict['orthorhombic_p'] = {}
for sg_num in orthorhombicPlist:
    # get TRIM k labels
    trim_dict['orthorhombic_p'][sg_num] = trim_k_labels(sg_num)
    print(sg_num, trim_dict['orthorhombic_p'][sg_num])

# Orthorhombic Centered C crystal system
orthorhombicClist = [ 63, 64]
trim_dict['orthorhombic_c'] = {}
for sg_num in orthorhombicClist:
    # get TRIM k labels
    trim_dict['orthorhombic_c'][sg_num] = trim_k_labels(sg_num)
    print(sg_num, trim_dict['orthorhombic_c'][sg_num])


# Tetragonal crystal system
tetralist= [127,128,129,130,135,136,137,138]
trim_dict['tetragonal'] = {}
for sg_num in tetralist:
    trim_dict['tetragonal'][sg_num] = trim_k_labels(sg_num)
    print(sg_num, trim_dict['tetragonal'][sg_num])

# Hexagonal crystal system
trim_dict['hexagonal'] = {
    176: ['L','H','A'], #C6h
    193: ['L','A'], #D6h 'H' is at the corner
    194: ['L','H','A'], #D6h
}

# Cubic crystal system
trim_dict['cubic'] = {
    205: ['X','M']# Th 'R' is not inversion symmetric
}

51 ['R', 'S', 'U', 'X']
52 ['R', 'S', 'T', 'Y']
53 ['R', 'T', 'U', 'Z']
54 ['R', 'S', 'U', 'X']
55 ['R', 'S', 'T', 'U', 'X', 'Y']
56 ['R', 'S', 'T', 'U', 'X', 'Y']
57 ['R', 'S', 'T', 'U', 'Y', 'Z']
58 ['R', 'S', 'T', 'U', 'X', 'Y']
59 ['R', 'S', 'T', 'U', 'X', 'Y']
60 ['R', 'S', 'T', 'U', 'X', 'Z']
61 ['R', 'S', 'T', 'U', 'X', 'Y', 'Z']
62 ['R', 'S', 'T', 'U', 'X', 'Y', 'Z']
63 ['R', 'T', 'Z']
64 ['R', 'S', 'T', 'Z']
127 ['R', 'X', 'A', 'M']
128 ['R', 'X', 'A', 'M']
129 ['R', 'X', 'A', 'M']
130 ['R', 'X', 'A', 'M']
135 ['R', 'X', 'A', 'M']
136 ['R', 'X', 'A', 'M']
137 ['R', 'X', 'A', 'M']
138 ['R', 'X', 'A', 'M']


In [62]:
momenta_dict = {} # momenta_dict[crystal_system][k_label] = k_vector
for crystal_system in trim_dict:
    momenta_dict[crystal_system] = {}
    for sg_num in trim_dict[crystal_system]:

        # get momenta dictionary for this space group
        sub_momenta_dict = get_momenta_dict(sg_num)

        # add momenta to momenta_dict
        for trim_k_label in trim_dict[crystal_system][sg_num]:

            if(trim_k_label in momenta_dict[crystal_system]):
                # check if k_label is unique for this crystal system
                if(sub_momenta_dict[trim_k_label] != momenta_dict[crystal_system][trim_k_label]):
                    print('Warning: k_label {} {} {} is not unique! {}'.format(trim_k_label, sg_num, crystal_system, sub_momenta_dict[trim_k_label]))
            else:
                momenta_dict[crystal_system][trim_k_label] = sub_momenta_dict[trim_k_label]

momenta_dict

{'monoclinic': {'C': ['1/2', '1/2', '0'],
  'D': ['0', '1/2', '1/2'],
  'E': ['1/2', '1/2', '1/2'],
  'Z': ['0', '1/2', '0']},
 'orthorhombic_p': {'R': ['1/2', '1/2', '1/2'],
  'S': ['1/2', '1/2', '0'],
  'U': ['1/2', '0', '1/2'],
  'X': ['1/2', '0', '0'],
  'T': ['0', '1/2', '1/2'],
  'Y': ['0', '1/2', '0'],
  'Z': ['0', '0', '1/2']},
 'orthorhombic_c': {'R': ['1/2', '1/2', '1/2'],
  'T': ['1', '0', '1/2'],
  'Z': ['0', '0', '1/2'],
  'S': ['1/2', '1/2', '0']},
 'tetragonal': {'R': ['0', '1/2', '1/2'],
  'X': ['0', '1/2', '0'],
  'A': ['1/2', '1/2', '1/2'],
  'M': ['1/2', '1/2', '0']},
 'hexagonal': {'L': ['1/2', '0', '1/2'],
  'H': ['1/3', '1/3', '1/2'],
  'A': ['0', '0', '1/2']},
 'cubic': {'X': ['0', '1/2', '0'], 'M': ['1/2', '1/2', '0']}}

# Group Theory Representation Algebra

## Parsing SG Irreps Table

In [43]:
# To map the string representation of a number to a float
number_format={
    '0': 0,
    '1': 1,
    '-1': -1,
    'i': 1j,
    '-i': -1j,
    '1/2': 0.5,
    '-1/2': -0.5,
    'i/2': 0.5j,
    '-i/2': -0.5j,
    '1/3': 1/3,
    '-1/3': -1/3,
    'i/3': 1j/3,
    '-i/3': -1j/3,
    '1/4': 0.25,
    '-1/4': -0.25,
    'i/4': 0.25j,
    '-i/4': -0.25j,
    '√3/2': np.sqrt(3)/2,
    '-√3/2': -np.sqrt(3)/2,
    '√3/2i': np.sqrt(3)/2j,
    '-√3/2i': -np.sqrt(3)/2j,
    '-(1+√3)/4': -(1+np.sqrt(3))/4,
    '(1+√3)/4': (1+np.sqrt(3))/4,
    '(1-√3)/4': -(1+np.sqrt(3))/4,
    '-(1-√3)/4': -(1-np.sqrt(3))/4,
    '(√3-1)/4': (np.sqrt(3)-1)/4,
    '-(√3-1)/4': -(np.sqrt(3)-1)/4,
    '(√3+1)/4': (np.sqrt(3)+1)/4,
    '-(√3+1)/4': -(np.sqrt(3)+1)/4,
}

In [86]:
pg_key_to_name = {
    '120101m010':'C2h:y',
    '120011m001':'C2h:z',
    '12001201021001m001m010m100':"D2h:xyz",
    '12001201022101m001m010m210':'D2h:pp',
    '120014+0014-00120102100211021101m0014+0014-001m010m100m110m110':'D4h',
    '13+0013-00120016-0016+00113+0013-001m0016-0016+001':'C6h',
    '13+0013-00120016-0016+00121102100201021102120221013+0013-001m0016-0016+001m110m100m010m110m120m210' : 'D6h',
    '12001201021003+1113+1113+1113+1113-1113-1113-1113-1111m001m010m1003+1113+1113+1113+1113-1113-1113-1113-111' : 'Th'
}

# Example of the Point Group key (unique string identifier for each point group)
rawdf=get_irreps_table(205,'R')
np.sum(np.array(rawdf['Refined Symbol']))

'12001201021003+1113+1113+1113+1113-1113-1113-1113-1111m001m010m1003+1113+1113+1113+1113-1113-1113-1113-111'

In [119]:
def check_point_group_symmetry(sg_num, k_label):
    """
    Checks what is the point group symmetry of a given group number and vector point.
    
    Args:
        sg_num (int): Number of the group.
        k_label (str): String representation of the momentum vector.
    
    Returns:
        str: The point group symmetry class.
    """
    df = get_irreps_table(sg_num, k_label)
    point_group_key = np.sum(np.array(df['Refined Symbol']))
    return pg_key_to_name[point_group_key]


def convert_bilbao_format(raw_df):
    """
    Converts the raw DataFrame from the Bilbao representation table to a more usable format.
    
    Args:
        raw_df (pd.DataFrame): Raw DataFrame from the Bilbao representation table.
    
    Returns:
        tuple: A tuple with (pg_name, index_list, column_list, irrep_table).
    """
    # Identify the point group
    point_group_key = np.sum(np.array(raw_df['Refined Symbol']))
    pg_name = pg_key_to_name[point_group_key]
    pg_name_trimmed = pg_name.split(':')[0]

    columnlist = []
    elementMap = {}
    
    
    ##### C2h
    if(pg_name_trimmed=="C2h"):
        columnlist = [ 'E' , 'C2(z)', 'i', 'σh' ]
        elementMap = { 'E' :[], 
                      'C2(z)':[2], 
                      'i':[3], 
                      'σh':[4]}
    
    ##### D2h
    elif(pg_name_trimmed=="D2h"):
        columnlist = ['E' , 'C2(z)', 'C2(y)', 'C2(x)', 'i', 'σ(xy)', 'σ(xz)', 'σ(yz)']
        elementMap = { 'E' : [] ,
                      'C2(z)': [2],
                      'C2(y)': [3],
                      'C2(x)': [4],
                      'i': [5],
                      'σ(xy)': [6],
                      'σ(xz)': [7],
                      'σ(yz)': [8]}
    ##### D4h
    elif(pg_name_trimmed=="D4h"):
        columnlist=[ 'E', '2C4(z)', 'C2', "2C'2", "2C''2" , "i", "2S4", "σh", "2σv", "2σd"]
        elementMap = { 'E':[], 
                      '2C4(z)':[3],
                      'C2':[2],
                      "2C'2":[6],
                      "2C''2":[7],
                      "i":[9],
                      "2S4":[12],
                      "σh":[10],
                      "2σv":[14],
                      "2σd":[15]}
        
    ##### C6h
    elif(pg_name_trimmed=="C6h"):
        columnlist=[ 'E', 'C6(z)', 'C3', 'C2', '(C3)^2', '(C6)^5', 'i', '(S3)^5', '(S6)^5', 'σh', 'S6', "S3"]
        elementMap = { 'E':[],
                      'C6(z)':[6],
                      'C3':[2],
                      'C2':[4],
                      '(C3)^2':[3],
                      '(C6)^5':[5],
                      'i':[7],
                      '(S3)^5':[9],
                      '(S6)^5':[11],
                      'σh':[10],
                      'S6':[12],
                      "S3":[8]}
    
    ##### D6h
    elif(pg_name_trimmed=="D6h"):
        columnlist=[ 'E', '2C6(z)', '2C3', 'C2', "3C'2", "3C''2", 'i', '2S3', '2S6', 'σh(xy)', '3σd', "3σv"]
        elementMap = { 'E':[],
                      '2C6(z)':[6],
                      '2C3':[2],
                      'C2':[4],
                      "3C'2":[7],
                      "3C''2":[10],
                      'i':[13],
                      '2S3':[14],
                      '2S6':[18],
                      'σh(xy)':[16],
                      '3σd':[19],
                      "3σv":[22]}

    ##### Th
    elif(pg_name_trimmed=="Th"):
        columnlist=[ 'E', '4C3', '4(C3)^2', '3C2', 'i', '4(S6)^5', '4S6', '3σh']
        elementMap = { 'E':[],
                      '4C3':[5],
                      '4(C3)^2':[9],
                      '3C2':[2],
                      'i':[13],
                      '4(S6)^5':[17],
                      '4S6':[21],
                      '3σh':[14]}
        
    else:
        raise("Point group not implemented") # type: ignore
    
    sg_irreps_names=np.array(raw_df.columns[3:])
    irreptable=[]
    for i in range(len(sg_irreps_names)):
        irreptable.append([None]*len(columnlist))

    for i in range(len(sg_irreps_names)):
        dim = int(np.sqrt(len(raw_df.loc[1,sg_irreps_names[i]].split())))
        #print(dim)

        # try:
        for j in range(len(columnlist)):
            
            glist = elementMap[columnlist[j]]
            irreptable[i][j]=np.eye(dim,dtype=float)

            for el in glist:
                sdata=raw_df.loc[el,sg_irreps_names[i]]
                sdata=sdata.split()
                #sdata=list(map(int,sdata))
                sdata=np.array(list(map(lambda x: number_format[x],sdata)),dtype=float)
                sdata=sdata.reshape((dim,dim))
                irreptable[i][j]=np.matmul(sdata,irreptable[i][j])
        # except KeyError:
        #     print("KeyError:"+indexlist[i])
        #     irreptable[i]=None
            
    return (pg_name,sg_irreps_names,columnlist,irreptable)

# Example usage
raw_df = get_irreps_table(129, 'M')
pg_name, index_list, column_list, irrep_table = convert_bilbao_format(raw_df)
print("PointGroup: ", pg_name)
print("IndexList: ", index_list)
print("ColumnList: ", column_list)

PointGroup:  D4h
IndexList:  ['M1' 'M2' 'M3' 'M4']
ColumnList:  ['E', '2C4(z)', 'C2', "2C'2", "2C''2", 'i', '2S4', 'σh', '2σv', '2σd']


## Direct Product

In [21]:
def direct_square(irrep_arr):
    """
    Direct square product of an irrep array and decompose into symmetric and antisymmetric components.
    
    Args:
        irrep_arr (list): List of matrices representing the irrep array.
    
    Returns:
        tuple: A tuple containing lists of matrices (symmetric, antisymmetric).
    """

    dim=len(irrep_arr)
    symarr = [None]*dim
    antarr = [None]*dim
    
    for i in range(dim):
        mat=irrep_arr[i]
        trsq=np.trace(mat)**2
        sqtr=np.trace(np.matmul(mat,mat))
        #print(trsq,sqtr)
        
        symtr=(trsq+sqtr)/2
        anttr=(trsq-sqtr)/2

        symarr[i]=int(round(symtr))
        antarr[i]=int(round(anttr))

        if(np.abs(symtr-symarr[i])>1e-5):
            print("Warning: Symmetric trace is not integer",symtr,symtr-symarr[i])
        if(np.abs(anttr-antarr[i])>1e-5):
            print("Warning: Antisymmetric trace is not integer")

    return (symarr,antarr)

## Irreducible Representations Decomposition (symmetry.jacobs-university.de)

In [90]:
pg_dict = {

    "C2h":{'code':502},
    "D2h":{'code':602},

    # C2h and D2h need to be divided into subgroups(look pg_key_to_name)
    'C2h:y': {'code': 502},
    'C2h:z': {'code': 502},
    'D2h:xyz': {'code': 602},
    'D2h:pp': {'code': 602},

    "D4h":{'code':604},
    "C6h":{'code':506},
    "D6h":{'code':606},
    "Th":{'code':901},

}

def get_decomposed_irreps(pg_name, li):
    """
    Get character table of a group and decompose into irreps.
    
    Args:
        pg_name (str): Point group name.
        li (list): List of integers (character table).
    
    Returns:
        pd.DataFrame: DataFrame of decomposed irreps.
    """
    
    url = 'http://symmetry.jacobs-university.de/cgi-bin/group.cgi'
    opts={
         'group':pg_dict[pg_name.split(':')[0]]['code'],
         'option': 1,
    }
    for i in range(len(li)):
        opts['a'+str(i+1)]=int(round(li[i]))
        
    htmlfile = get_html_file(url,opts)
    
    df_nl = pd.read_html(htmlfile)
    
    if(len(df_nl)<2):
        #There is no decomposition
        return None

    return df_nl[1]

# Example usage
df_nl=get_decomposed_irreps("D2h",[3.0, 1.0, -1.0, 1.0, 1.0, 3.0, 1.0, -1.0])
df_nl

,Ag,B1g,B2g,B3g,Au,B1u,B2u,B3u
0,1,1,0,0,0,0,0,1


In [146]:
def get_product_irreps(pg_name,sg_irreps_names,irreptable):
    """
    Get product irreps of a point group.

    Args:
        pg_name (str): Point group name.
        sg_irreps_names (list): List of sg irreps names.
        irreptable (list): List of irrep arrays.

    Returns:
        dict: Dictionary of product irreps.
    """
    
    decomp={}
    for i in range(len(sg_irreps_names)):
        
        if(irreptable[i] is None):
            continue

        decomp[sg_irreps_names[i]]={}
        symarr, antarr = direct_square(irreptable[i])

        decomp[sg_irreps_names[i]]['sym']=symarr
        decomp[sg_irreps_names[i]]['ant']=antarr

        expression =""
        # symmetric part
        df_nl=get_decomposed_irreps(pg_name,symarr)
        if(type(df_nl)==pd.core.frame.DataFrame):
            for rep in list(df_nl):
                if df_nl[rep][0]:
                    degenstr= "" if df_nl[rep][0]==1 else str(df_nl[rep][0])
                    expression+=degenstr+rep+" + "
        else:
            expression+="None"

        # antisymmetric part
        #expression += "["
        df_nl=get_decomposed_irreps(pg_name,antarr)
        if(type(df_nl)==pd.core.frame.DataFrame):
            for rep in list(df_nl):
                if df_nl[rep][0]:
                    degenstr= "" if df_nl[rep][0]==1 else str(df_nl[rep][0])
                    expression+=degenstr+"["+rep+"] + "
        else:
            expression+="None"
        #expression += "]"
        expression=expression[:-3]

        decomp[sg_irreps_names[i]]['exp']=expression

        # Get the pg info
        decomp[sg_irreps_names[i]]['pg_info'] = [ pg_name , list(df_nl) ]

        print(sg_irreps_names[i],"^2 = ",expression)
        
    return decomp

# Example usage
rawdf = get_irreps_table(129,'M')
pg_name,indexlist,columnlist,irreptable = convert_bilbao_format(rawdf)
decomp=get_product_irreps(pg_name,indexlist,irreptable)
decomp

M1 ^2 =  A1g + B2g + B2u + [A1u]
M2 ^2 =  A1g + B2g + B2u + [A1u]
M3 ^2 =  A1g + B2g + A2u + [B1u]
M4 ^2 =  A1g + B2g + A2u + [B1u]


{'M1': {'sym': [3, -1, 3, -1, 3, 1, 1, 1, 1, 1],
  'ant': [1, 1, 1, 1, 1, -1, -1, -1, -1, -1],
  'exp': 'A1g + B2g + B2u + [A1u]',
  'pg_info': ['D4h',
   ['A1g', 'A2g', 'B1g', 'B2g', 'Eg', 'A1u', 'A2u', 'B1u', 'B2u', 'Eu']]},
 'M2': {'sym': [3, -1, 3, -1, 3, 1, 1, 1, 1, 1],
  'ant': [1, 1, 1, 1, 1, -1, -1, -1, -1, -1],
  'exp': 'A1g + B2g + B2u + [A1u]',
  'pg_info': ['D4h',
   ['A1g', 'A2g', 'B1g', 'B2g', 'Eg', 'A1u', 'A2u', 'B1u', 'B2u', 'Eu']]},
 'M3': {'sym': [3, 1, 3, -1, 1, 1, -1, 1, 1, 3],
  'ant': [1, -1, 1, 1, -1, -1, 1, -1, -1, 1],
  'exp': 'A1g + B2g + A2u + [B1u]',
  'pg_info': ['D4h',
   ['A1g', 'A2g', 'B1g', 'B2g', 'Eg', 'A1u', 'A2u', 'B1u', 'B2u', 'Eu']]},
 'M4': {'sym': [3, 1, 3, -1, 1, 1, -1, 1, 1, 3],
  'ant': [1, -1, 1, 1, -1, -1, 1, -1, -1, 1],
  'exp': 'A1g + B2g + A2u + [B1u]',
  'pg_info': ['D4h',
   ['A1g', 'A2g', 'B1g', 'B2g', 'Eg', 'A1u', 'A2u', 'B1u', 'B2u', 'Eu']]}}

In [94]:
direct_square(irreptable[2])

([3, 1, 3, -1, 1, 1, -1, 1, 1, 3], [1, -1, 1, 1, -1, -1, 1, -1, -1, 1])

## Get kp theories from TRIM

In [123]:
def get_kp_theory(sg_num, k_point):
    """
    Get the product of irreducible representations (irreps) for a given group number and k-point.

    Parameters:
    sg_num (int): The group number.
    k_point (str): The k-point name.

    Returns:
    dict: A dictionary containing the product irreps.

    Example:
    get_kp_theory(129, 'M')
    """

    raw_df = get_irreps_table(sg_num, k_point)
    point_group, index_list, column_list, irrep_table = convert_bilbao_format(raw_df)
    decomp = get_product_irreps(point_group, index_list, irrep_table)
    
    return decomp
#get_kp_theory(205,'R')
get_kp_theory(205,'X')
#get_kp_theory(205,'M')
#get_kp_theory(11,'C')

['X1' 'X2']
X1 ^2 =  Ag + B1g + B3u + [B2u]
X2 ^2 =  Ag + B1g + B3u + [B2u]


{'X1': {'sym': [3, 1, -1, 1, 1, 3, 1, -1],
  'ant': [1, -1, 1, -1, -1, 1, -1, 1],
  'exp': 'Ag + B1g + B3u + [B2u]',
  'pg_info': ['D2h:xyz',
   ['Ag', 'B1g', 'B2g', 'B3g', 'Au', 'B1u', 'B2u', 'B3u']]},
 'X2': {'sym': [3, 1, -1, 1, 1, 3, 1, -1],
  'ant': [1, -1, 1, -1, -1, 1, -1, 1],
  'exp': 'Ag + B1g + B3u + [B2u]',
  'pg_info': ['D2h:xyz',
   ['Ag', 'B1g', 'B2g', 'B3g', 'Au', 'B1u', 'B2u', 'B3u']]}}

In [156]:
data={}

def save(data,file_name="data.json"):
    # Serializing json
    json_object = json.dumps(data, indent=2)

    # Writing to sample.json
    with open(file_name, "w") as outfile:
        outfile.write(json_object)
        
def iterscan(momenta_dict,file_name="data.json"):

    if(not os.path.isfile(file_name)):
        save({},file_name)

    with open(file_name, 'r') as openfile:
        # Reading from json file
        data = json.load(openfile)
    
    for sg_num in momenta_dict:
        print("Space Group Number ",sg_num)
        for k_label in momenta_dict[sg_num]:
            print("K-point Label ",k_label)
            newd=get_kp_theory(sg_num,k_label)
            data[str(sg_num)+":"+str(k_label)]=newd

    save(data,file_name)
        
    return data

In [157]:
trim_dict.keys()

dict_keys(['monoclinic', 'orthorhombic_p', 'orthorhombic_c', 'tetragonal', 'hexagonal', 'cubic'])

### Monoclinic crystal system

In [158]:
# Monoclinic crystal system
data=iterscan(
    trim_dict['monoclinic']
)

Space Group Number  11
K-point Label  C
C1 ^2 =  Ag + Bg + Bu + [Au]
K-point Label  D
D1 ^2 =  Ag + Bg + Bu + [Au]
K-point Label  E
E1 ^2 =  Ag + Bg + Bu + [Au]
K-point Label  Z
Z1 ^2 =  Ag + Bg + Bu + [Au]
Space Group Number  14
K-point Label  D
D1+D2+ ^2 =  Ag + 2Bg + [Ag]
D1-D2- ^2 =  Ag + 2Bg + [Ag]
K-point Label  Z
Z1 ^2 =  Ag + Bg + Bu + [Au]


### Orthorhombic crystal system

In [159]:
# Orthorhombic crystal system
data=iterscan(
    trim_dict['orthorhombic_p']
)
data=iterscan(
    trim_dict['orthorhombic_c']
)

Space Group Number  51
K-point Label  R
R1 ^2 =  Ag + B2g + B1u + [B3u]
R2 ^2 =  Ag + B2g + B1u + [B3u]
K-point Label  S
S1 ^2 =  Ag + B2g + B1u + [B3u]
S2 ^2 =  Ag + B2g + B1u + [B3u]
K-point Label  U
U1 ^2 =  Ag + B2g + B1u + [B3u]
U2 ^2 =  Ag + B2g + B1u + [B3u]
K-point Label  X
X1 ^2 =  Ag + B2g + B1u + [B3u]
X2 ^2 =  Ag + B2g + B1u + [B3u]
Space Group Number  52
K-point Label  R
R1 ^2 =  Ag + B3g + B1u + [B2u]
R2 ^2 =  Ag + B3g + B1u + [B2u]
K-point Label  S
S1S2 ^2 =  Ag + 2B1g + B2g + 2B3g + 2Au + B1u + B3u + [Ag] + [B2g] + [B1u] + 2[B2u] + [B3u]
K-point Label  T
T1+ ^2 =  Ag + B1g + B3g + [B2g]
T1- ^2 =  Ag + B1g + B3g + [B2g]
K-point Label  Y
Y1 ^2 =  Ag + B1g + B3u + [B2u]
Y2 ^2 =  Ag + B1g + B3u + [B2u]
Space Group Number  53
K-point Label  R
R1+ ^2 =  Ag + B2g + B3g + [B1g]
R1- ^2 =  Ag + B2g + B3g + [B1g]
K-point Label  T
T1 ^2 =  Ag + B3g + B2u + [B1u]
T2 ^2 =  Ag + B3g + B2u + [B1u]
K-point Label  U
U1+ ^2 =  Ag + B2g + B3g + [B1g]
U1- ^2 =  Ag + B2g + B3g + [B1g]
K-poin

### Tetragonal crystal system

In [160]:
# Tetragonal crystal system
data=iterscan(
    trim_dict['tetragonal']
)

Space Group Number  127
K-point Label  R
R1 ^2 =  Ag + B1g + B3u + [B2u]
R2 ^2 =  Ag + B1g + B3u + [B2u]
K-point Label  X
X1 ^2 =  Ag + B1g + B3u + [B2u]
X2 ^2 =  Ag + B1g + B3u + [B2u]
K-point Label  A
A1+A4+ ^2 =  A1g + 2B2g + [A1g]
A1-A4- ^2 =  A1g + 2B2g + [A1g]
A2+A3+ ^2 =  A1g + 2B2g + [A1g]
A2-A3- ^2 =  A1g + 2B2g + [A1g]
A5+ ^2 =  A1g + A2g + B2g + [B1g]
A5- ^2 =  A1g + A2g + B2g + [B1g]
K-point Label  M
M1+M4+ ^2 =  A1g + 2B2g + [A1g]
M1-M4- ^2 =  A1g + 2B2g + [A1g]
M2+M3+ ^2 =  A1g + 2B2g + [A1g]
M2-M3- ^2 =  A1g + 2B2g + [A1g]
M5+ ^2 =  A1g + A2g + B2g + [B1g]
M5- ^2 =  A1g + A2g + B2g + [B1g]
Space Group Number  128
K-point Label  R
R1+ ^2 =  Ag + B1g + B3g + [B2g]
R1- ^2 =  Ag + B1g + B3g + [B2g]
K-point Label  X
X1 ^2 =  Ag + B1g + B3u + [B2u]
X2 ^2 =  Ag + B1g + B3u + [B2u]
K-point Label  A
A1 ^2 =  A1g + A2g + B2u + [B1u]
A2 ^2 =  A1g + A2g + B2u + [B1u]
A3A4 ^2 =  A1g + A2g + 2B1g + 2B2g + A1u + A2u + 2B2u + [A1g] + [A2g] + [A1u] + [A2u] + 2[B1u]
K-point Label  M
M1+M4

### Hexagonal crystal system

In [161]:
# Hexagonal crystal system
data=iterscan(
    trim_dict['hexagonal']
)

Space Group Number  176
K-point Label  L
L1 ^2 =  Ag + Bg + Bu + [Au]
K-point Label  H
H1H2 ^2 =  3Ag + 3Bg + Au + 3Bu + [Ag] + [Bg] + 3[Au] + [Bu]
H3H6 ^2 =  Ag + Bg + E1g + E2g + Au + Bu + E1u + [Ag] + [Bg] + [Au] + [Bu] + [E2u]
H4H5 ^2 =  Ag + Bg + E1g + E2g + Au + Bu + E1u + [Ag] + [Bg] + [Au] + [Bu] + [E2u]
K-point Label  A
A1 ^2 =  Ag + Bg + Bu + [Au]
A2A3 ^2 =  Ag + Bg + E1g + E2g + Au + Bu + E1u + [Ag] + [Bg] + [Au] + [Bu] + [E2u]
Space Group Number  193
K-point Label  L
L1 ^2 =  Ag + B3g + B2u + [B1u]
L2 ^2 =  Ag + B3g + B2u + [B1u]
K-point Label  A
A1 ^2 =  A1g + B2g + B1u + [A2u]
A2 ^2 =  A1g + B2g + B1u + [A2u]
A3 ^2 =  A1g + B2g + E1g + E2g + A1u + B1u + E1u + [A2g] + [B1g] + [A2u] + [B2u] + [E2u]
Space Group Number  194
K-point Label  L
L1 ^2 =  Ag + B2g + B3u + [B1u]
L2 ^2 =  Ag + B2g + B3u + [B1u]
K-point Label  H
H1 ^2 =  A1g + B1g + E1g + E2g + A1u + B2u + E1u + [A2g] + [B2g] + [A2u] + [B1u] + [E2u]
H2 ^2 =  A1g + B1g + E1g + E2g + A1u + B2u + E1u + [A2g] + [B2g] + [A

### Cubic crystal system

In [162]:
# Cubic crystal system
data=iterscan({
    205:['X','M']#,'R'], #Th
})

Space Group Number  205
K-point Label  X
X1 ^2 =  Ag + B1g + B3u + [B2u]
X2 ^2 =  Ag + B1g + B3u + [B2u]
K-point Label  M
M1M2 ^2 =  Ag + 2B1g + 2B2g + B3g + Au + 2B1u + B3u + [Ag] + [B3g] + [Au] + 2[B2u] + [B3u]


In [163]:
print(json.dumps(data, indent=2)[:1000])

{
  "11:C": {
    "C1": {
      "sym": [
        3,
        -1,
        1,
        1
      ],
      "ant": [
        1,
        1,
        -1,
        -1
      ],
      "exp": "Ag + Bg + Bu + [Au]",
      "pg_info": [
        "C2h:y",
        [
          "Ag",
          "Bg",
          "Au",
          "Bu"
        ]
      ]
    }
  },
  "11:D": {
    "D1": {
      "sym": [
        3,
        -1,
        1,
        1
      ],
      "ant": [
        1,
        1,
        -1,
        -1
      ],
      "exp": "Ag + Bg + Bu + [Au]",
      "pg_info": [
        "C2h:y",
        [
          "Ag",
          "Bg",
          "Au",
          "Bu"
        ]
      ]
    }
  },
  "11:E": {
    "E1": {
      "sym": [
        3,
        -1,
        1,
        1
      ],
      "ant": [
        1,
        1,
        -1,
        -1
      ],
      "exp": "Ag + Bg + Bu + [Au]",
      "pg_info": [
        "C2h:y",
        [
          "Ag",
          "Bg",
          "Au",
          "Bu"
        ]
      ]
    

# (Legacy)Point Group Properties

## Momentum Dependence

In [ ]:
pgtok = {
    'C2h':{
        'Ag': 'x^2-y^2,xy,z^2',
        'Bg': 'xz,yz',
        'Au': 'z',
        'Bu': 'x,y'
    },
    'D2h':{
        'Ag': 'x^2-y^2,z^2',
        'B1g': 'xy',
        'B2g': 'xz',
        'B3g': 'yz',
        'Au': 'xyz',
        'B1u': 'z',
        'B2u': 'y',
        'B3u': 'x'
    },
    'D4h':{
        'A1g': 'z^2',
        'A2g': 'xy(x^2-y^2)',
        'B1g': 'x^2-y^2',
        'B2g': 'xy',
        'Eg': '{xz,yz}',
        'A1u': 'xyz(x^2-y^2)',
        'A2u': 'z',
        'B1u': 'xyz',
        'B2u': 'z(x^2-y^2)',
        'Eu': '{x,y}'
    },
    'C6h':{
        'Ag':'z^2',
        'Bg':'xz(x^2-3y^2)',
        'E1g':'{xz,yz}',
        'E2g':'{x^2-y^2,xy}',
        'Au':'z',
        'Bu':'x(x^2-3y^2)',
        'E1u':'{x,y}',
        'E2u':'{z(x^2-y^2),xyz}'
    },
    'D6h':{
        'A1g': 'z^2',
        'A2g': 'xy(x^2-3y^2)(2x^2-y^2)',
        'B1g': 'yz(3x^2-y^2)',
        'B2g': 'xz(x^2-3y^2)',
        'E1g': '{xz,yz}',
        'E2g': '{x^2-y^2,xy}',
        'A1u': 'None',
        'A2u': 'z',
        'B1u': 'x(x^2-3y^2)',
        'B2u': 'y(3x^2-y^2)',
        'E1u': '{x,y}',
        'E2u': '{z(x^2-y^2),xyz}'
    }
}

In [ ]:
spintorep = {
    'C2h':{
        'sx,sy': 'Bg',
        'sz': 'Ag',
    },
    'D2h':{
        'sx': 'B3g',
        'sy': 'B2g',
        'sz': 'B1g',
    },
    'D4h':{
        '{sx,sy}': 'Eg',
        'sz': 'A2g',
    },
    'C6h':{
        '{sx,sy}': 'E1g',
        'sz': 'Ag',
    },
    'D6h':{
        '{sx,sy}': 'E1g',
        'sz': 'A2g'
    }
}

In [119]:
invMap = {}
eightfold = {}
for d in data:
    itr=data[d]
    for rep in itr:
        repp=itr[rep]['exp']
        if(len(repp.split('+'))>4 or repp=='None[None]'):
            eightfold[d+":"+rep]=repp 
        elif(repp in invMap):
            invMap[repp].append([d,rep])
        else:
            invMap[repp]=[[d,rep]]

In [120]:
eightfold

{'52:S:S1S2': 'Ag + 2B1g + B2g + 2B3g + 2Au + B1u + B3u + [Ag] + [B2g] + [B1u] + 2[B2u] + [B3u]',
 '60:T:T1T2': 'Ag + B1g + 2B2g + 2B3g + 2Au + B2u + B3u + [Ag] + [B1g] + 2[B1u] + [B2u] + [B3u]',
 '60:U:U1U2': 'Ag + B1g + 2B2g + 2B3g + Au + B1u + 2B2u + [Ag] + [B1g] + [Au] + [B1u] + 2[B3u]',
 '60:R:R1R2': 'Ag + 2B1g + 2B2g + B3g + Au + 2B2u + B3u + [Ag] + [B3g] + [Au] + 2[B1u] + [B3u]',
 '62:S:S1S2': 'Ag + 2B1g + 2B2g + B3g + Au + 2B1u + B3u + [Ag] + [B3g] + [Au] + 2[B2u] + [B3u]',
 '62:R:R1R2': 'Ag + B1g + 2B2g + 2B3g + 2B1u + B2u + B3u + [Ag] + [B1g] + 2[Au] + [B2u] + [B3u]',
 '128:A:A3A4': 'A1g + A2g + 2B1g + 2B2g + A1u + A2u + 2B2u + [A1g] + [A2g] + [A1u] + [A2u] + 2[B1u]',
 '205:M:M1M2': 'Ag + 2B1g + 2B2g + B3g + Au + 2B1u + B3u + [Ag] + [B3g] + [Au] + 2[B2u] + [B3u]',
 '54:U:U1U2': 'Ag + 2B1g + 2B2g + B3g + 2Au + B1u + B2u + [Ag] + [B3g] + [B1u] + [B2u] + 2[B3u]',
 '54:R:R1R2': 'Ag + 2B1g + 2B2g + B3g + 2Au + B1u + B2u + [Ag] + [B3g] + [B1u] + [B2u] + 2[B3u]',
 '56:U:U1U2': 'Ag +

In [121]:
invMap

{'Ag + 2Bg + [Ag]': [['14:D', 'D1+D2+'],
  ['14:D', 'D1-D2-'],
  ['64:R', 'R1+R2+'],
  ['64:R', 'R1-R2-']],
 'Ag + Bg + Bu + [Au]': [['14:Z', 'Z1'],
  ['11:C', 'C1'],
  ['11:D', 'D1'],
  ['11:E', 'E1'],
  ['11:Z', 'Z1'],
  ['176:L', 'L1'],
  ['63:R', 'R1'],
  ['176:A', 'A1']],
 'Ag + B1g + B3u + [B2u]': [['52:Y', 'Y1'],
  ['52:Y', 'Y2'],
  ['55:Y', 'Y1'],
  ['55:Y', 'Y2'],
  ['58:Y', 'Y1'],
  ['58:Y', 'Y2'],
  ['62:Y', 'Y1'],
  ['62:Y', 'Y2'],
  ['127:X', 'X1'],
  ['127:X', 'X2'],
  ['128:X', 'X1'],
  ['128:X', 'X2'],
  ['135:X', 'X1'],
  ['135:X', 'X2'],
  ['136:X', 'X1'],
  ['136:X', 'X2'],
  ['205:X', 'X1'],
  ['205:X', 'X2'],
  ['57:S', 'S1'],
  ['57:S', 'S2'],
  ['57:Y', 'Y1'],
  ['57:Y', 'Y2'],
  ['61:Y', 'Y1'],
  ['61:Y', 'Y2'],
  ['127:R', 'R1'],
  ['127:R', 'R2'],
  ['55:T', 'T1'],
  ['55:T', 'T2'],
  ['135:R', 'R1'],
  ['135:R', 'R2']],
 'Ag + B3g + B1u + [B2u]': [['52:R', 'R1'],
  ['52:R', 'R2'],
  ['129:X', 'X1'],
  ['129:X', 'X2'],
  ['56:Y', 'Y1'],
  ['56:Y', 'Y2'],
  ['5

In [71]:
len(invMap['None[None]'])

24

In [39]:
type2dict={}
for rep in list(invMap):
    if(rep[-2]=='g'):
        
        #Symmetry
        latrep=rep.replace("g", "_g").replace("u", "_u")
        latrep=latrep.replace("1_g", "_{ig}").replace("2_g", "_{ig}").replace("3_g", "_{ig}")
        latrep=latrep.replace("1_u", "_{iu}").replace("2_u", "_{iu}").replace("3_u", "_{iu}")
        
        if (not latrep in type2dict):
                type2dict[latrep]=[]
        type2dict[latrep]=type2dict[latrep]+invMap[rep]

In [48]:
latexstr=""
for rep in list(type2dict):
    #Symmetry
    latexstr+="\("+rep+"\) &\n"

    #Hamiltonian
    
    #latham = rep.split(" + ")
    #pointGp ='D2h' if latham[0]=='Ag' else 'D4h'
    #if pointGp == 'D2h':
    #    tau1 = d2hmntDep[latham[1]]
    #    tau2 = d2hmntDep
    #    latexstr+=" \( H = \epsilon + " + latham.join(" + ")+ "\) &\n"
    if rep in hamrep:
        latexstr+=hamrep[rep]
    else:
        latexstr+=" ... "
    latexstr+="&\n"
    
    #Space Group Momenta
    itermap={}
    for a,b in type2dict[rep]:
        sgnum=a.split(":")[0]
        if sgnum in itermap:
            itermap[sgnum].append(b)
        else:
            itermap[sgnum]=[b]
    for sgnum in itermap:
        latexstr += sgnum + "("
        latexstr +=', '.join(itermap[sgnum])
        latexstr += "), "

    latexstr=latexstr[:-2]+"\\\ \hline\n"
print(latexstr)

\(A_g + B_g + [A_g]\) &
\begin{equation}
H=\epsilon+k_2 k_\perp\tau_1+k_2 k_\perp\tau_2+\tau_3(k_2 k_\perp\sigma_1+\sigma_2+k_y k_\perp\sigma_3)
\end{equation}&
14(D1+D2+, D1-D2-), 64(R1+R2+, R1-R2-)\\ \hline
\(A_g + B_{ig} + B_{ig} + [B_{ig}]\) &
 ... &
52(T1+, T1-), 58(T1+, T1-), 128(R1+, R1-), 136(R1+, R1-), 53(U1+, U1-, R1+, R1-), 60(S1+, S1-)\\ \hline
\(A_g + B_{ig} + [A_g]\) &
 ... &
55(S1+S2+, S1-S2-, S3+S4+, S3-S4-), 58(S1+S2+, S1-S2-, S3+S4+, S3-S4-), 56(R1+R2+, R1-R2-, R3+R4+, R3-R4-), 62(U1+U4+, U1-U4-, U2+U3+, U2-U3-)\\ \hline
\(A_{ig} + B_{ig} + [A_{ig}]\) &
 ... &
127(M1+M4+, M1-M4-, M2+M3+, M2-M3-, A1+A4+, A1-A4-, A2+A3+, A2-A3-), 128(M1+M4+, M1-M4-, M2+M3+, M2-M3-), 135(M1+M4+, M1-M4-, M2+M3+, M2-M3-), 136(M1+M4+, M1-M4-, M2+M3+, M2-M3-), 138(A1+A4+, A1-A4-, A2+A3+, A2-A3-)\\ \hline
\(A_{ig} + A_{ig} + B_{ig} + [B_{ig}]\) &
 ... &
127(M5+, M5-, A5+, A5-), 128(M5+, M5-), 135(M5+, M5-), 136(M5+, M5-), 138(A5+, A5-)\\ \hline



In [47]:
hamrep={
    'A_g + B_g + [A_g]':'\\begin{equation}\nH=\\epsilon+k_2 k_\\perp\\tau_1+k_2 k_\\perp\\tau_2+\\tau_3(k_2 k_\\perp\\sigma_1+\\sigma_2+k_y k_\\perp\\sigma_3)\n\\end{equation}',
    'A_g + B_{ig} + B_{ig} + [B_{ig}]':'\\begin{equation}\nH=\\epsilon+k_2 k_\\perp\\tau_1+k_2 k_\\perp\\tau_2+\\tau_3(k_2 k_\\perp\\sigma_1+\\sigma_2+k_y k_\\perp\\sigma_3)\n\\end{equation}',
    'A_g + B_g + [A_g]':'\\begin{equation}\nH=\\epsilon+k_2 k_\\perp\\tau_1+k_2 k_\\perp\\tau_2+\\tau_3(k_2 k_\\perp\\sigma_1+\\sigma_2+k_y k_\\perp\\sigma_3)\n\\end{equation}'
}

In [49]:
type1dict={}
for rep in list(invMap):
    if(rep[-2]=='u'):
        
        #Symmetry
        latrep=rep.replace("g", "_g").replace("u", "_u")
        latrep=latrep.replace("1_g", "_{ig}").replace("2_g", "_{ig}").replace("3_g", "_{ig}")
        latrep=latrep.replace("1_u", "_{iu}").replace("2_u", "_{iu}").replace("3_u", "_{iu}")
        
        if (not latrep in type1dict):
                type1dict[latrep]=[]
        type1dict[latrep]=type1dict[latrep]+invMap[rep]

In [51]:
latexstr=""
for rep in list(type1dict):
    #Symmetry
    latexstr+="\("+rep+"\) &\n"

    #Hamiltonian
    
    #latham = rep.split(" + ")
    #pointGp ='D2h' if latham[0]=='Ag' else 'D4h'
    #if pointGp == 'D2h':
    #    tau1 = d2hmntDep[latham[1]]
    #    tau2 = d2hmntDep
    #    latexstr+=" \( H = \epsilon + " + latham.join(" + ")+ "\) &\n"
    if rep in hamrep:
        latexstr+=hamrep[rep]
    else:
        latexstr+=" ... "
    latexstr+="&\n"
    
    #Space Group Momenta
    itermap={}
    for a,b in type1dict[rep]:
        sgnum=a.split(":")[0]
        if sgnum in itermap:
            itermap[sgnum].append(b)
        else:
            itermap[sgnum]=[b]
    for sgnum in itermap:
        latexstr += sgnum + "("
        latexstr +=', '.join(itermap[sgnum])
        latexstr += "), "

    latexstr=latexstr[:-2]+"\\\ \hline\n"
print(latexstr)

\(A_g + B_g + B_u + [A_u]\) &
 ... &
14(Z1), 11(C1, D1, E1, Z1), 176(L1), 63(R1)\\ \hline
\(A_g + B_{ig} + B_{iu} + [B_{iu}]\) &
 ... &
52(Y1, Y2, R1, R2), 55(Y1, Y2), 58(Y1, Y2), 62(Y1, Y2, Z1, Z2), 127(X1, X2, R1, R2), 128(X1, X2), 135(X1, X2), 136(X1, X2), 205(X1, X2), 57(S1, S2, Y1, Y2, Z1, Z2, U1, U2), 61(Y1, Y2, Z1, Z2, X1, X2), 129(X1, X2, R1, R2), 56(Y1, Y2, X1, X2), 59(T1, T2, Y1, Y2, X1, X2, U1, U2), 130(X1, X2), 137(R1, R2, X1, X2), 138(X1, X2), 53(Z1, Z2), 60(Z1, Z2), 64(T1, T2, Z1, Z2), 63(T1, T2, Z1, Z2), 51(X1, X2, S1, S2, U1, U2, R1, R2), 54(X1, X2, S1, S2)\\ \hline
\(A_g + B_{ig} + B_{iu} + [A_u]\) &
 ... &
58(R1, R2), 56(S1, S2), 59(S1, S2, R1, R2), 62(T1, T2)\\ \hline
\(A_{ig} + A_{ig} + B_{iu} + [B_{iu}]\) &
 ... &
128(A1, A2), 137(A1, A2)\\ \hline
\(A_{ig} + B_{ig} + B_{iu} + [A_{iu}]\) &
 ... &
129(M1, M2, A1, A2), 136(A3, A4), 130(M1, M2), 137(M1, M2), 138(M1, M2)\\ \hline
\(A_{ig} + B_{ig} + A_{iu} + [B_{iu}]\) &
 ... &
129(M3, M4, A3, A4), 136(A1, A2), 130(M3, 

In [76]:
reu=""
for i in invMap['None[None]']:
    aa,bb =i[0].split(":")
    reu+=aa+"("+bb+")"
reu

'52(S)\\n60(T)\\n60(U)\\n60(R)\\n62(S)\\n62(R)\\n128(A)\\n205(M)\\n54(U)\\n54(R)\\n56(U)\\n56(T)\\n57(T)\\n57(R)\\n61(T)\\n61(S)\\n61(U)\\n61(R)\\n61(R)\\n130(R)\\n130(A)\\n130(A)\\n137(A)\\n138(R)\\n'

# Point Group Classification

In [ ]:
def checkPointGroupSymmetry(sg_num,k_label):
    

In [49]:
checkPointGroupSymmetry(14,'D')

'C2h:y'

In [48]:
pgdata = {}
for sgstr in data.keys():
    sgnum = int(sgstr.split(":")[0])
    kpoint= sgstr.split(":")[1]
    pointGroup = checkPointGroupSymmetry(sgnum,kpoint)
    pointGroup = pointGroup[0] + "_{" + pointGroup[1:] + "}"
    print(sgstr,pointGroup)
    pgdata[sgstr]=pointGroup

14:D C_{2h:y}
14:Z C_{2h:y}
52:S D_{2h:xyz}
52:Y D_{2h:xyz}
52:R D_{2h:xyz}
52:T D_{2h:xyz}
53:Z D_{2h:xyz}
53:U D_{2h:xyz}
53:R D_{2h:xyz}
55:Y D_{2h:xyz}
55:S D_{2h:xyz}
58:Y D_{2h:xyz}
58:T D_{2h:xyz}
58:S D_{2h:xyz}
58:R D_{2h:xyz}
60:Z D_{2h:xyz}
60:T D_{2h:xyz}
60:S D_{2h:xyz}
60:U D_{2h:xyz}
60:R D_{2h:xyz}
62:Z D_{2h:xyz}
62:T D_{2h:xyz}
62:S D_{2h:xyz}
62:U D_{2h:xyz}
62:R D_{2h:xyz}
62:Y D_{2h:xyz}
127:X D_{2h:xyz}
127:M D_{4h}
128:X D_{2h:xyz}
128:M D_{4h}
128:R D_{2h:xyz}
128:A D_{4h}
129:X D_{2h:xyz}
129:M D_{4h}
135:X D_{2h:xyz}
135:M D_{4h}
136:X D_{2h:xyz}
136:M D_{4h}
136:R D_{2h:xyz}
136:A D_{4h}
205:X D_{2h:xyz}
205:M D_{2h:xyz}
11:C C_{2h:y}
11:D C_{2h:y}
11:E C_{2h:y}
11:Z C_{2h:y}
51:X D_{2h:xyz}
51:S D_{2h:xyz}
51:U D_{2h:xyz}
51:R D_{2h:xyz}
54:X D_{2h:xyz}
54:S D_{2h:xyz}
54:U D_{2h:xyz}
54:R D_{2h:xyz}
56:X D_{2h:xyz}
56:S D_{2h:xyz}
56:U D_{2h:xyz}
56:R D_{2h:xyz}
56:T D_{2h:xyz}
56:Y D_{2h:xyz}
57:Z D_{2h:xyz}
57:T D_{2h:xyz}
57:S D_{2h:xyz}
57:U D_{2h:xyz}


In [261]:
invMap = {}
eightfold = {}
for d in data:
    itr=data[d]
    pg = pgdata[d]
    for rep in itr:
        repp="("+pg+")"+itr[rep]['exp']
        if(len(repp.split('+'))>4 or repp.split(')')[1]=='None[None]'):
            eightfold[d+":"+rep]=repp
        elif(repp in invMap):
            invMap[repp].append([d,rep])
        else:
            invMap[repp]=[[d,rep]]

In [262]:
invMap

{'(C_{2h:y})Ag + 2Bg + [Ag]': [['14:D', 'D1+D2+'], ['14:D', 'D1-D2-']],
 '(C_{2h:y})Ag + Bg + Bu + [Au]': [['14:Z', 'Z1'],
  ['11:C', 'C1'],
  ['11:D', 'D1'],
  ['11:E', 'E1'],
  ['11:Z', 'Z1']],
 '(D_{2h:xyz})Ag + B1g + B3u + [B2u]': [['52:Y', 'Y1'],
  ['52:Y', 'Y2'],
  ['55:Y', 'Y1'],
  ['55:Y', 'Y2'],
  ['58:Y', 'Y1'],
  ['58:Y', 'Y2'],
  ['62:Y', 'Y1'],
  ['62:Y', 'Y2'],
  ['127:X', 'X1'],
  ['127:X', 'X2'],
  ['128:X', 'X1'],
  ['128:X', 'X2'],
  ['135:X', 'X1'],
  ['135:X', 'X2'],
  ['136:X', 'X1'],
  ['136:X', 'X2'],
  ['205:X', 'X1'],
  ['205:X', 'X2'],
  ['57:S', 'S1'],
  ['57:S', 'S2'],
  ['57:Y', 'Y1'],
  ['57:Y', 'Y2'],
  ['61:Y', 'Y1'],
  ['61:Y', 'Y2'],
  ['127:R', 'R1'],
  ['127:R', 'R2'],
  ['55:T', 'T1'],
  ['55:T', 'T2'],
  ['135:R', 'R1'],
  ['135:R', 'R2']],
 '(D_{2h:xyz})Ag + B3g + B1u + [B2u]': [['52:R', 'R1'],
  ['52:R', 'R2'],
  ['129:X', 'X1'],
  ['129:X', 'X2'],
  ['56:Y', 'Y1'],
  ['56:Y', 'Y2'],
  ['59:T', 'T1'],
  ['59:T', 'T2'],
  ['59:Y', 'Y1'],
  ['59:Y'

In [263]:
eightfold

{'52:S:S1S2': '(D_{2h:xyz})Ag + 2B1g + B2g + 2B3g + 2Au + B1u + B3u + [Ag] + [B2g] + [B1u] + 2[B2u] + [B3u]',
 '60:T:T1T2': '(D_{2h:xyz})Ag + B1g + 2B2g + 2B3g + 2Au + B2u + B3u + [Ag] + [B1g] + 2[B1u] + [B2u] + [B3u]',
 '60:U:U1U2': '(D_{2h:xyz})Ag + B1g + 2B2g + 2B3g + Au + B1u + 2B2u + [Ag] + [B1g] + [Au] + [B1u] + 2[B3u]',
 '60:R:R1R2': '(D_{2h:xyz})Ag + 2B1g + 2B2g + B3g + Au + 2B2u + B3u + [Ag] + [B3g] + [Au] + 2[B1u] + [B3u]',
 '62:S:S1S2': '(D_{2h:xyz})Ag + 2B1g + 2B2g + B3g + Au + 2B1u + B3u + [Ag] + [B3g] + [Au] + 2[B2u] + [B3u]',
 '62:R:R1R2': '(D_{2h:xyz})Ag + B1g + 2B2g + 2B3g + 2B1u + B2u + B3u + [Ag] + [B1g] + 2[Au] + [B2u] + [B3u]',
 '128:A:A3A4': '(D_{4h})A1g + A2g + 2B1g + 2B2g + A1u + A2u + 2B2u + [A1g] + [A2g] + [A1u] + [A2u] + 2[B1u]',
 '205:M:M1M2': '(D_{2h:xyz})Ag + 2B1g + 2B2g + B3g + Au + 2B1u + B3u + [Ag] + [B3g] + [Au] + 2[B2u] + [B3u]',
 '54:U:U1U2': '(D_{2h:xyz})Ag + 2B1g + 2B2g + B3g + 2Au + B1u + B2u + [Ag] + [B3g] + [B1u] + [B2u] + 2[B3u]',
 '54:R:R1R2':

In [318]:
ty1map={}
ty2map={}
for sgstr in invMap.keys():
    if sgstr[-2]=='u':
        ty1map[sgstr]=invMap[sgstr].copy()
    else:
        ty2map[sgstr]=invMap[sgstr].copy()

In [319]:
ty1map

{'(C_{2h:y})Ag + Bg + Bu + [Au]': [['14:Z', 'Z1'],
  ['11:C', 'C1'],
  ['11:D', 'D1'],
  ['11:E', 'E1'],
  ['11:Z', 'Z1']],
 '(D_{2h:xyz})Ag + B1g + B3u + [B2u]': [['52:Y', 'Y1'],
  ['52:Y', 'Y2'],
  ['55:Y', 'Y1'],
  ['55:Y', 'Y2'],
  ['58:Y', 'Y1'],
  ['58:Y', 'Y2'],
  ['62:Y', 'Y1'],
  ['62:Y', 'Y2'],
  ['127:X', 'X1'],
  ['127:X', 'X2'],
  ['128:X', 'X1'],
  ['128:X', 'X2'],
  ['135:X', 'X1'],
  ['135:X', 'X2'],
  ['136:X', 'X1'],
  ['136:X', 'X2'],
  ['205:X', 'X1'],
  ['205:X', 'X2'],
  ['57:S', 'S1'],
  ['57:S', 'S2'],
  ['57:Y', 'Y1'],
  ['57:Y', 'Y2'],
  ['61:Y', 'Y1'],
  ['61:Y', 'Y2'],
  ['127:R', 'R1'],
  ['127:R', 'R2'],
  ['55:T', 'T1'],
  ['55:T', 'T2'],
  ['135:R', 'R1'],
  ['135:R', 'R2']],
 '(D_{2h:xyz})Ag + B3g + B1u + [B2u]': [['52:R', 'R1'],
  ['52:R', 'R2'],
  ['129:X', 'X1'],
  ['129:X', 'X2'],
  ['56:Y', 'Y1'],
  ['56:Y', 'Y2'],
  ['59:T', 'T1'],
  ['59:T', 'T2'],
  ['59:Y', 'Y1'],
  ['59:Y', 'Y2'],
  ['130:X', 'X1'],
  ['130:X', 'X2'],
  ['137:R', 'R1'],
  ['13

In [320]:
ty2map

{'(C_{2h:y})Ag + 2Bg + [Ag]': [['14:D', 'D1+D2+'], ['14:D', 'D1-D2-']],
 '(D_{2h:xyz})Ag + B1g + B3g + [B2g]': [['52:T', 'T1+'],
  ['52:T', 'T1-'],
  ['58:T', 'T1+'],
  ['58:T', 'T1-'],
  ['128:R', 'R1+'],
  ['128:R', 'R1-'],
  ['136:R', 'R1+'],
  ['136:R', 'R1-']],
 '(D_{2h:xyz})Ag + B2g + B3g + [B1g]': [['53:U', 'U1+'],
  ['53:U', 'U1-'],
  ['53:R', 'R1+'],
  ['53:R', 'R1-']],
 '(D_{2h:xyz})Ag + 2B1g + [Ag]': [['55:S', 'S1+S2+'],
  ['55:S', 'S1-S2-'],
  ['55:S', 'S3+S4+'],
  ['55:S', 'S3-S4-'],
  ['58:S', 'S1+S2+'],
  ['58:S', 'S1-S2-'],
  ['58:S', 'S3+S4+'],
  ['58:S', 'S3-S4-'],
  ['56:R', 'R1+R2+'],
  ['56:R', 'R1-R2-'],
  ['56:R', 'R3+R4+'],
  ['56:R', 'R3-R4-'],
  ['55:R', 'R1+R2+'],
  ['55:R', 'R1-R2-'],
  ['55:R', 'R3+R4+'],
  ['55:R', 'R3-R4-']],
 '(D_{2h:xyz})Ag + B1g + B2g + [B3g]': [['60:S', 'S1+'],
  ['60:S', 'S1-'],
  ['58:U', 'U1+'],
  ['58:U', 'U1-']],
 '(D_{2h:xyz})Ag + 2B2g + [Ag]': [['62:U', 'U1+U4+'],
  ['62:U', 'U1-U4-'],
  ['62:U', 'U2+U3+'],
  ['62:U', 'U2-U3-']

In [321]:
pgdata['176:A']

'C_{6h}'

In [322]:
def convertToDict(targetdict,sourcedict,key,permutstr="",delopt=True):
    if(key not in sourcedict):
        print("Key not found in source dictionary : ",key)
        return targetdict
    array = list(sourcedict[key])
    for el in array:
        sgnum, kpoint = el[0].split(":")
        reps = el[1].split(kpoint)
        reps = [ kpoint + "_{" + i[0] + "}" + ("^{"+i[1]+"}" if len(i)>1 else "") for i in reps[1:]]
        rep = "".join(reps)

        if len(permutstr)>0:
            rep = rep+permutstr
            
        if sgnum in targetdict:
            if(kpoint in targetdict[sgnum]):
                targetdict[sgnum][kpoint].append(rep)
            else:
                targetdict[sgnum][kpoint]=[rep]
        else:
            targetdict[sgnum]={kpoint:[rep]}
    
    if(delopt):
        del(sourcedict[key])

    return targetdict

In [340]:
def convertToLatex(dict):
    array=list(dict.keys())
    #sort array
    array.sort(key=lambda x: int(x))
    for sgnum in array:
        latexstr = "\("+ sgnum + "("
        for kpoint in dict[sgnum]:
            
            reps=dict[sgnum][kpoint]
            
            ind=0
            while(ind+1<len(reps)):
                if(reps[ind].replace("1","2")==reps[ind+1]):
                    reps[ind]=reps[ind].replace("1","1,2")
                    reps.pop(ind+1)
                elif(reps[ind].replace("+","?").replace("-","+").replace("?","-")==reps[ind+1]):
                    reps[ind]=reps[ind].replace("+","\pm").replace("-","\mp")
                    reps.pop(ind+1)
                elif(reps[ind].replace("3","4")==reps[ind+1]):
                    reps[ind]=reps[ind].replace("3","3,4")
                    reps.pop(ind+1)
                
                ind+=1
            
            latexstr+=(", ").join(reps)+", "

        latexstr=latexstr[:-2]+")\), \n"
        print(latexstr)

C2h 1

In [324]:
del(ty1map['(C_{2h:y})Ag + Bg + Bu + [Au]'])
del(ty1map['(C_{2h:z})Ag + Bg + Bu + [Au]'])

C2h 2

In [325]:
print(ty2map['(C_{2h:y})Ag + 2Bg + [Ag]'])
del(ty2map['(C_{2h:y})Ag + 2Bg + [Ag]'])
print(ty2map['(C_{2h:z})Ag + 2Bg + [Ag]'])
del(ty2map['(C_{2h:z})Ag + 2Bg + [Ag]'])

[['14:D', 'D1+D2+'], ['14:D', 'D1-D2-']]
[['64:R', 'R1+R2+'], ['64:R', 'R1-R2-']]


D2h 1-1

In [326]:
d2h11 = {}
convertToDict(d2h11,ty1map,'(D_{2h:xyz})Ag + B2g + B1u + [B3u]')

convertToDict(d2h11,ty1map,'(D_{2h:xyz})Ag + B2g + B3u + [B1u]','(xz)')#'x\leftrightarrow z')
convertToDict(d2h11,ty1map,'(D_{2h:xyz})Ag + B1g + B2u + [B3u]','(yz)')#'y\leftrightarrow z')
convertToDict(d2h11,ty1map,'(D_{2h:xyz})Ag + B3g + B1u + [B2u]','(xy)')#'x\leftrightarrow y')

convertToDict(d2h11,ty1map,'(D_{2h:xyz})Ag + B1g + B3u + [B2u]','(xyz)')#'\cycr')
convertToDict(d2h11,ty1map,'(D_{2h:xyz})Ag + B3g + B2u + [B1u]','(zyx)')#'\cycl')


{'51': {'X': ['X_{1}', 'X_{2}'],
  'S': ['S_{1}', 'S_{2}'],
  'U': ['U_{1}', 'U_{2}'],
  'R': ['R_{1}', 'R_{2}']},
 '54': {'X': ['X_{1}', 'X_{2}'], 'S': ['S_{1}', 'S_{2}']},
 '56': {'X': ['X_{1}', 'X_{2}'], 'Y': ['Y_{1}(xy)', 'Y_{2}(xy)']},
 '59': {'X': ['X_{1}', 'X_{2}'],
  'U': ['U_{1}', 'U_{2}'],
  'T': ['T_{1}(xy)', 'T_{2}(xy)'],
  'Y': ['Y_{1}(xy)', 'Y_{2}(xy)']},
 '61': {'X': ['X_{1}', 'X_{2}'],
  'Y': ['Y_{1}(xyz)', 'Y_{2}(xyz)'],
  'Z': ['Z_{1}(zyx)', 'Z_{2}(zyx)']},
 '60': {'X': ['X_{1}', 'X_{2}'], 'Z': ['Z_{1}(zyx)', 'Z_{2}(zyx)']},
 '62': {'X': ['X_{1}', 'X_{2}'],
  'Z': ['Z_{1}(xz)', 'Z_{2}(xz)'],
  'Y': ['Y_{1}(xyz)', 'Y_{2}(xyz)']},
 '55': {'U': ['U_{1}(yz)', 'U_{2}(yz)'],
  'X': ['X_{1}(yz)', 'X_{2}(yz)'],
  'Y': ['Y_{1}(xyz)', 'Y_{2}(xyz)'],
  'T': ['T_{1}(xyz)', 'T_{2}(xyz)']},
 '58': {'X': ['X_{1}(yz)', 'X_{2}(yz)'], 'Y': ['Y_{1}(xyz)', 'Y_{2}(xyz)']},
 '52': {'R': ['R_{1}(xy)', 'R_{2}(xy)'], 'Y': ['Y_{1}(xyz)', 'Y_{2}(xyz)']},
 '129': {'X': ['X_{1}(xy)', 'X_{2}(xy)']

In [327]:
convertToLatex(d2h11)

\(51(X_{1,2}, S_{1,2}, U_{1,2}, R_{1,2})\), 

\(52(R_{1,2}(xy), Y_{1,2}(xyz))\), 

\(53(Z_{1,2}(zyx), T_{1,2}(zyx))\), 

\(54(X_{1,2}, S_{1,2})\), 

\(55(U_{1,2}(yz), X_{1,2}(yz), Y_{1,2}(xyz), T_{1,2}(xyz))\), 

\(56(X_{1,2}, Y_{1,2}(xy))\), 

\(57(S_{1,2}(xyz), Y_{1,2}(xyz), Z_{1,2}(zyx), U_{1,2}(zyx))\), 

\(58(X_{1,2}(yz), Y_{1,2}(xyz))\), 

\(59(X_{1,2}, U_{1,2}, T_{1,2}(xy), Y_{1,2}(xy))\), 

\(60(X_{1,2}, Z_{1,2}(zyx))\), 

\(61(X_{1,2}, Y_{1,2}(xyz), Z_{1,2}(zyx))\), 

\(62(X_{1,2}, Z_{1,2}(xz), Y_{1,2}(xyz))\), 

\(63(T_{1,2}(zyx), Z_{1,2}(zyx))\), 

\(64(T_{1,2}(zyx), Z_{1,2}(zyx))\), 

\(127(X_{1,2}(xyz), R_{1,2}(xyz))\), 

\(128(X_{1,2}(xyz))\), 

\(129(X_{1,2}(xy), R_{1,2}(xy))\), 

\(130(X_{1,2}(xy))\), 

\(135(X_{1,2}(xyz), R_{1,2}(xyz))\), 

\(136(X_{1,2}(xyz))\), 

\(137(R_{1,2}(xy), X_{1,2}(xy))\), 

\(138(X_{1,2}(xy))\), 

\(205(X_{1,2}(xyz))\), 



D2h 1-2

In [328]:
d2h12 = {}
convertToDict(d2h12,ty1map,'(D_{2h:xyz})Ag + B1g + B1u + [Au]',"")
convertToDict(d2h12,ty1map,'(D_{2h:xyz})Ag + B2g + B2u + [Au]',"(yz)")
convertToDict(d2h12,ty1map,'(D_{2h:xyz})Ag + B3g + B3u + [Au]',"(xz)")

Key not found in source dictionary :  (D_{2h:xyz})Ag + B2g + B2u + [Au]


{'58': {'R': ['R_{1}', 'R_{2}']},
 '56': {'S': ['S_{1}', 'S_{2}']},
 '59': {'S': ['S_{1}', 'S_{2}'], 'R': ['R_{1}', 'R_{2}']},
 '62': {'T': ['T_{1}(xz)', 'T_{2}(xz)']}}

In [329]:
convertToLatex(d2h12)

\(56(S_{1,2})\), 

\(58(R_{1,2})\), 

\(59(S_{1,2}, R_{1,2})\), 

\(62(T_{1,2}(xz))\), 



D2h 2-1

In [330]:
d2h21 = {}
delopt=True
convertToDict(d2h21,ty2map,'(D_{2h:xyz})Ag + B1g + B3g + [B2g]','',delopt)

convertToDict(d2h21,ty2map,'(D_{2h:xyz})Ag + B3g + B1g + [B2g]','(xz)',delopt)
convertToDict(d2h21,ty2map,'(D_{2h:xyz})Ag + B1g + B2g + [B3g]','(xy)',delopt)
convertToDict(d2h21,ty2map,'(D_{2h:xyz})Ag + B2g + B3g + [B1g]','(yz)',delopt)

convertToDict(d2h21,ty2map,'(D_{2h:xyz})Ag + B3g + B2g + [B1g]','(xyz)',delopt)
convertToDict(d2h21,ty2map,'(D_{2h:xyz})Ag + B2g + B1g + [B3g]','(zyx)',delopt)

Key not found in source dictionary :  (D_{2h:xyz})Ag + B3g + B1g + [B2g]
Key not found in source dictionary :  (D_{2h:xyz})Ag + B3g + B2g + [B1g]
Key not found in source dictionary :  (D_{2h:xyz})Ag + B2g + B1g + [B3g]


{'52': {'T': ['T_{1}^{+}', 'T_{1}^{-}']},
 '58': {'T': ['T_{1}^{+}', 'T_{1}^{-}'],
  'U': ['U_{1}^{+}(xy)', 'U_{1}^{-}(xy)']},
 '128': {'R': ['R_{1}^{+}', 'R_{1}^{-}']},
 '136': {'R': ['R_{1}^{+}', 'R_{1}^{-}']},
 '60': {'S': ['S_{1}^{+}(xy)', 'S_{1}^{-}(xy)']},
 '53': {'U': ['U_{1}^{+}(yz)', 'U_{1}^{-}(yz)'],
  'R': ['R_{1}^{+}(yz)', 'R_{1}^{-}(yz)']}}

In [331]:
convertToLatex(d2h21)

\(52(T_{1}^{\pm})\), 

\(53(U_{1}^{\pm}(yz), R_{1}^{\pm}(yz))\), 

\(58(T_{1}^{\pm}, U_{1}^{\pm}(xy))\), 

\(60(S_{1}^{\pm}(xy))\), 

\(128(R_{1}^{\pm})\), 

\(136(R_{1}^{\pm})\), 



D2h 2-2

In [332]:
d2h22 = {}
delopt=True
convertToDict(d2h22,ty2map,'(D_{2h:xyz})Ag + 2B1g + [Ag]','',delopt)

convertToDict(d2h21,ty2map,'(D_{2h:xyz})Ag + 2B2g + [Ag]','(xz)',delopt)
convertToDict(d2h21,ty2map,'(D_{2h:xyz})Ag + 2B3g + [Ag]','(yz)',delopt)

Key not found in source dictionary :  (D_{2h:xyz})Ag + 2B3g + [Ag]


{'52': {'T': ['T_{1}^{\\pm}']},
 '58': {'T': ['T_{1}^{\\pm}'], 'U': ['U_{1}^{\\pm}(xy)']},
 '128': {'R': ['R_{1}^{\\pm}']},
 '136': {'R': ['R_{1}^{\\pm}']},
 '60': {'S': ['S_{1}^{\\pm}(xy)']},
 '53': {'U': ['U_{1}^{\\pm}(yz)'], 'R': ['R_{1}^{\\pm}(yz)']},
 '62': {'U': ['U_{1}^{+}U_{4}^{+}(xz)',
   'U_{1}^{-}U_{4}^{-}(xz)',
   'U_{2}^{+}U_{3}^{+}(xz)',
   'U_{2}^{-}U_{3}^{-}(xz)']}}

In [333]:
convertToLatex(d2h22)

\(55(S_{1}^{\pm}S_{2}^{\pm}, S_{3}^{\pm}S_{4}^{\pm}, R_{1}^{\pm}R_{2}^{\pm}, R_{3}^{\pm}R_{4}^{\pm})\), 

\(56(R_{1}^{\pm}R_{2}^{\pm}, R_{3}^{\pm}R_{4}^{\pm})\), 

\(58(S_{1}^{\pm}S_{2}^{\pm}, S_{3}^{\pm}S_{4}^{\pm})\), 



D4h 1-1

In [334]:
d4h11 = {}
delopt=True
convertToDict(d4h11,ty1map,'(D_{4h})A1g + A2g + B2u + [B1u]','',delopt)

{'128': {'A': ['A_{1}', 'A_{2}']}, '137': {'A': ['A_{1}', 'A_{2}']}}

In [335]:
convertToLatex(d4h11)

\(128(A_{1,2})\), 

\(137(A_{1,2})\), 



D4h 1-2

In [342]:
d4h12 = {}
delopt=True
convertToDict(d4h12,ty1map,'(D_{4h})A1g + B2g + B2u + [A1u]','',delopt)

{'129': {'M': ['M_{1}', 'M_{2}'], 'A': ['A_{1}', 'A_{2}']},
 '136': {'A': ['A_{3}', 'A_{4}']},
 '130': {'M': ['M_{1}', 'M_{2}']},
 '137': {'M': ['M_{1}', 'M_{2}']},
 '138': {'M': ['M_{1}', 'M_{2}']}}

In [343]:
convertToLatex(d4h12)

\(129(M_{1,2}, A_{1,2})\), 

\(130(M_{1,2})\), 

\(136(A_{3,4})\), 

\(137(M_{1,2})\), 

\(138(M_{1,2})\), 



D4h 1-3

In [348]:
d4h13 = {}
delopt=True
convertToDict(d4h13,ty1map,'(D_{4h})A1g + B2g + A2u + [B1u]','',delopt)
convertToLatex(d4h13)

\(129(M_{3,4}, A_{3,4})\), 

\(130(M_{3,4})\), 

\(136(A_{1,2})\), 

\(137(M_{3,4})\), 

\(138(M_{3,4})\), 



In [349]:
ty1map

{'(C_{2h:z})Ag + Bg + Au + [Bu]': [['64:S', 'S1']],
 '(C_{6h})Ag + Bg + Bu + [Au]': [['176:A', 'A1']],
 '(D_{2h:pp})Ag + B3g + B2u + [B1u]': [['193:L', 'L1'], ['193:L', 'L2']],
 '(D_{6h})A1g + B2g + B1u + [A2u]': [['193:A', 'A1'], ['193:A', 'A2']],
 '(D_{2h:pp})Ag + B2g + B3u + [B1u]': [['194:L', 'L1'], ['194:L', 'L2']],
 '(D_{6h})A1g + B1g + B2u + [A2u]': [['194:A', 'A1'], ['194:A', 'A2']]}

In [401]:
# Read latexchunks/type1.txt file

classmap={
    "$C_{2h}^{(1,1)}$":"(C_{2h})^{I}_{1}",
    "$C_{2h}^{(2)}$":"(C_{2h})^{II}_{2}",

    "$D_{2h}^{(1,1)}$":"(D_{2h})^{I}_{3}",
    "$D_{2h}^{(1,2)}$":"(D_{2h})^{I}_{1}",
    "$D_{2h}^{(2,1)}$":"(D_{2h})^{II}_{4}",
    "$D_{2h}^{(2,2)}$":"(D_{2h})^{II}_{2}",
    
    "$D_{4h}^{(1,1)}$":"(D_{4h})^{I}_{5}",
    "$D_{4h}^{(1,2)}$":"(D_{4h})^{I}_{1}",
    "$D_{4h}^{(1,3)}$":"(D_{4h})^{I}_{3}",
    "$D_{4h}^{(2,1)}$":"(D_{4h})^{II}_{2}",
    "$D_{4h}^{(2,2)}$":"(D_{4h})^{II}_{4}",

    "$C_{6h}^{(1)}$":"(C_{6h})^{I}_{1}",
    "$D_{6h}^{(1)}$":"(D_{6h})^{I}_{1}",
}

outfile = ""
fname="latexchunks/type1.txt"
oname="latexchunks/type1out.tex"
with open(fname) as f:
    lines = f.readlines()
    isHam = False
    reptxt = ""
    for line in lines:
        if(line[0]=="$"):
            #Class line
            outfile+=line[:-2]+"\\\\ \hline \n"
            print(line[:-2]+"\\\\ \hline")
        elif(line[:3]=="\(H"):
            #Hamiltonian line
            isHam = True
            outfile+="\multicolumn{2}{|c|}{" + line[:-1] +"}\\\\ \n"
        elif(isHam):
            #Hamiltonian SOC(second) line
            outfile+="\multicolumn{2}{|c|}{" + line[:-2]+"}\\\\ \hline"
            isHam = False
            reptxt = ""
        elif(line[:9]=="\\\\ \hline"):
            #End of table
            if(reptxt!=""):
                outfile+="\n \multicolumn{2}{|c|}{"+reptxt+"}\\\\ "
                reptxt = ""
            outfile+='\hline \hline \n \n'
        else :
            #Momenta lines
            if(len(reptxt)>30):
                outfile+="\n \multicolumn{2}{|c|}{"+reptxt+"}\\\\ "
                reptxt = ""
            if(reptxt!=""):
                reptxt+=", "
            reptxt += line[:-1]
            if(reptxt[-1]==" "):
                reptxt = reptxt[:-1]
            if(reptxt[-1]==","):
                reptxt = reptxt[:-1]
            else:
                print(reptxt)

with open(oname, "w") as f:
    f.write(outfile)

$C_{2h}^{(1,1)}$ & \(A_g + B_g + [A_u]+ B_u\) \\ \hline
\(176(L_1(yz))\), \(63(R_1(yz))\)
$C_{2h}^{(1,2)}$ & \(A_g + B_{g} + [B_u] + A_u\) \\ \hline
\(64(S_1)\)
$D_{2h}^{(1,1)}$ & \(A_g + B_{2g} + [B_{1u}] + B_{3u}\) \\ \hline
\(138(X_{1,2}(xy))\), \(\bm{193(L_{1,2}(xy)(x\rightarrow \frac{2x+y}{\sqrt{3}}))}\)
\(\bm{194(L_{1,2}(x\rightarrow \frac{2x+y}{\sqrt{3}}))}\)
\(205(X_{1,2}(xyz))\)
$D_{2h}^{(1,2)}$ & \(A_g + B_{1g} + [A_u] + B_{1u} \) \\ \hline
\(59(S_{1,2}, R_{1,2})\), \(62(T_{1,2}(xz))\)
$D_{4h}^{(1,1)}$ & \(A_{1g} + A_{2g} + [B_{1u}] + B_{2u}\) \\ \hline
\(128(A_{1,2})\), \(137(A_{1,2})\)
$D_{4h}^{(1,2)}$ & \(A_{1g} + B_{2g} + [A_{1u}] + B_{2u}\) \\ \hline
\(138(M_{1,2})\)
$D_{4h}^{(1,3)}$ & \(A_{1g} + B_{2g} + [B_{1u}] + A_{2u}\) \\ \hline
\(138(M_{3,4})\)
$C_{6h}^{(1)}$ & \(A_{g} + B_{g} + [A_{u}] + B_{u}\) \\ \hline
\(176(A_1)\)
$D_{6h}^{(1)}$ & \(A_{1g} + B_{2g} + [A_{2u}] + B_{1u} \) \\ \hline
\(193(A_{1,2})\), \(194(A_{1,2}(xy))\)


In [382]:
ty2map

{'(D_{4h})A1g + 2B2g + [A1g]': [['127:M', 'M1+M4+'],
  ['127:M', 'M1-M4-'],
  ['127:M', 'M2+M3+'],
  ['127:M', 'M2-M3-'],
  ['128:M', 'M1+M4+'],
  ['128:M', 'M1-M4-'],
  ['128:M', 'M2+M3+'],
  ['128:M', 'M2-M3-'],
  ['135:M', 'M1+M4+'],
  ['135:M', 'M1-M4-'],
  ['135:M', 'M2+M3+'],
  ['135:M', 'M2-M3-'],
  ['136:M', 'M1+M4+'],
  ['136:M', 'M1-M4-'],
  ['136:M', 'M2+M3+'],
  ['136:M', 'M2-M3-'],
  ['138:A', 'A1+A4+'],
  ['138:A', 'A1-A4-'],
  ['138:A', 'A2+A3+'],
  ['138:A', 'A2-A3-'],
  ['127:A', 'A1+A4+'],
  ['127:A', 'A1-A4-'],
  ['127:A', 'A2+A3+'],
  ['127:A', 'A2-A3-']],
 '(D_{4h})A1g + A2g + B2g + [B1g]': [['127:M', 'M5+'],
  ['127:M', 'M5-'],
  ['128:M', 'M5+'],
  ['128:M', 'M5-'],
  ['135:M', 'M5+'],
  ['135:M', 'M5-'],
  ['136:M', 'M5+'],
  ['136:M', 'M5-'],
  ['138:A', 'A5+'],
  ['138:A', 'A5-'],
  ['127:A', 'A5+'],
  ['127:A', 'A5-']]}

D4h 2-1

In [385]:
d4h21 = {}
delopt=True
convertToDict(d4h21,ty2map,'(D_{4h})A1g + 2B2g + [A1g]','',delopt)
convertToLatex(d4h21)

\(127(M_{1}^{\pm}M_{4}^{\pm}, M_{2}^{\pm}M_{3}^{\pm}, A_{1}^{\pm}A_{4}^{\pm}, A_{2}^{\pm}A_{3}^{\pm})\), 

\(128(M_{1}^{\pm}M_{4}^{\pm}, M_{2}^{\pm}M_{3}^{\pm})\), 

\(135(M_{1}^{\pm}M_{4}^{\pm}, M_{2}^{\pm}M_{3}^{\pm})\), 

\(136(M_{1}^{\pm}M_{4}^{\pm}, M_{2}^{\pm}M_{3}^{\pm})\), 

\(138(A_{1}^{\pm}A_{4}^{\pm}, A_{2}^{\pm}A_{3}^{\pm})\), 



D4h 2-2

In [388]:
d4h22 = {}
delopt=True
convertToDict(d4h22,ty2map,'(D_{4h})A1g + A2g + B2g + [B1g]','',delopt)
convertToLatex(d4h22)

\(127(M_{5}^{\pm}, A_{5}^{\pm})\), 

\(128(M_{5}^{\pm})\), 

\(135(M_{5}^{\pm})\), 

\(136(M_{5}^{\pm})\), 

\(138(A_{5}^{\pm})\), 



In [390]:
ty2map

{}

In [17]:
#Read latexchunks/combinedout.tex file
kptypedict = {}
isalreadyin = set({})
kptypedict2 = {}
with open("latexchunks/table2.tex") as f: #"latexchunks/combinedout.tex") as f:
    lines = f.readlines()
    kptype=""
    sgnum=0
    for line in lines:
        if(line[0]=="$" or line[:2]==" $"):
            #Class line
            kptype=line.split("&")[0].replace(" ","")
            print(kptype)
        else:
            #Momenta lines
            for kpt in line.split("\(")[1:]:
                if(kpt[0]!="H" and kpt[0]!="~"):
                    sgnum=kpt.split("(")[0]
                    kstr = sgnum + ":" + kptype
                    
                    if(kstr in isalreadyin):
                        print(kstr)
                    elif(sgnum in kptypedict):
                        isalreadyin.add(kptype)
                        kptypedict[sgnum]+=", "+kptype
                    else:
                        isalreadyin.add(kstr)
                        kptypedict[sgnum]=kptype
                    
                    # kreps=kpt.split("(")[1].split(")")[0].split(", ")
                    # for krep in kreps:
                    #     kstr = sgnum + ":" + krep[0]
                    #     if(not(sgnum in kptypedict2)):
                    #         kptypedict2[sgnum]=krep[1:]
                    #     else:
                    #         kptypedict2[sgnum]={kstr:krep[1:]}




with open("8foldlist.dat") as f:
    lines = f.readlines()
    checklist = []
    for line in lines:
        sgnum = line.split(':')[0][2:]
        if(sgnum in checklist):
            continue
        checklist.append(sgnum)
        if(sgnum in kptypedict):
            if(sgnum=="193"):
                print(sgnum + " " + kptypedict[sgnum])
                print(line)
            kptypedict[sgnum]+=", 8-fold"
        else:
            kptypedict[sgnum]="8-fold"

$C_{2h,1}^{\text{type}1}$
$C_{2h,2}^{\text{type}2}$
64:$C_{2h,2}^{\text{type}2}$
$D_{2h,1}^{\text{type}1}$
$D_{2h,2}^{\text{type}2}$
$D_{2h,3}^{\text{type}1}$
$D_{2h,4}^{\text{type}2}$
$D_{4h,1}^{\text{type}1}$
$D_{4h,2}^{\text{type}2}$
$D_{4h,3}^{\text{type}1}$
$D_{4h,4}^{\text{type}2}$
$D_{4h,5}^{\text{type}1}$
$C_{6h}^{\text{type}1}$
$D_{6h}^{\text{type}1}$
193 $D_{2h,3}^{\text{type}1}$, $D_{6h}^{\text{type}1}$
 '193:A:A3': 'A1g + B2g + E1g + E2g + A1u + B1u + E1u + [A2g] + [B1g] + [A2u] + [B2u] + [E2u]',



In [18]:
kptypedict

{'11': '$C_{2h,1}^{\\text{type}1}$',
 '14': '$C_{2h,1}^{\\text{type}1}$, $C_{2h,2}^{\\text{type}2}$',
 '63': '$C_{2h,1}^{\\text{type}1}$, $D_{2h,3}^{\\text{type}1}$',
 '176': '$C_{2h,1}^{\\text{type}1}$, $C_{6h}^{\\text{type}1}$, 8-fold',
 '64': '$C_{2h,2}^{\\text{type}2}$, $D_{2h,3}^{\\text{type}1}$',
 'A_g + B_{g} + [B_u] + A_u\\) \\\\ \\hline \n': '$C_{2h,2}^{\\text{type}2}$',
 '+\\tau_3[': '$C_{2h,2}^{\\text{type}2}$',
 '56': '$D_{2h,1}^{\\text{type}1}$, $D_{2h,2}^{\\text{type}2}$, $D_{2h,3}^{\\text{type}1}$, 8-fold',
 '58': '$D_{2h,1}^{\\text{type}1}$, $D_{2h,2}^{\\text{type}2}$, $D_{2h,3}^{\\text{type}1}$, $D_{2h,4}^{\\text{type}2}$',
 '59': '$D_{2h,1}^{\\text{type}1}$, $D_{2h,3}^{\\text{type}1}$',
 '62': '$D_{2h,1}^{\\text{type}1}$, $D_{2h,3}^{\\text{type}1}$, 8-fold',
 '55': '$D_{2h,2}^{\\text{type}2}$, $D_{2h,3}^{\\text{type}1}$',
 '51': '$D_{2h,3}^{\\text{type}1}$',
 '52': '$D_{2h,3}^{\\text{type}1}$, $D_{2h,4}^{\\text{type}2}$, 8-fold',
 '53': '$D_{2h,3}^{\\text{type}1}$, $D

In [51]:
outfile = ""
with open("latexchunks/table1.tex") as f:
    lines = f.readlines()
    for line in lines:
        if(len(line)>10):
            outfile+=line[:-3]
            outfile+="&\n"
            outfile+=kptypedict[line.split("&")[1]]
            outfile+="\\\\ \n"
        else:
            outfile+=line

###Add multirows
# lines = outfile.split("\n")
# crystaltype = ""
# index = 0
# count = 0
# for i in range(len(lines)):
#     line = lines[i]
#     if(line[0]!="$" and line[0]!="\\"):
#         cols = line.split("&")
#         lines[i] = "&".join([""]+cols[1:])
#         if(cols[0].strip()!=crystaltype):
#             if(count>0):
#                 lines[index] = "\\multirow{"+str(count)+"}{*}{"+crystaltype+"}"+lines[index]
#             count = 1
#             index = i
#             crystaltype = cols[0].strip()
#         else:
#             count+=1

#         print(line)
# outfile = "\n".join(lines)

with open("latexchunks/table1out.tex", "w") as f:
    f.write(outfile)

8-fold list

In [479]:
table1_arr=[]
with open("latexchunks/table1.tex") as f:
    lines = f.readlines()
    for line in lines:
        if(len(line)>10):
            table1_arr.append(line.split("&"))

for i in range(len(table1_arr)):
    planeinfo=table1_arr[i][3].split('$')[1]
    planeinfo=map(lambda x : x.replace("(","").replace(")","").split(",") , planeinfo.split('),('))
    table1_arr[i][3] = list(planeinfo)

table1_arr

[['Monoclinic $(C_{2h})$ ', '11', ' $P2_1/m$', [['u', '1/2', 'w']]],
 ['Monoclinic $(C_{2h})$', '14', '$P2_1/c$', [['u', '1/2', 'w']]],
 ['Orthorhombic $(D_{2h})$', '51', '$Pmma$', [['1/2', 'v', 'w']]],
 ['Orthorhombic $(D_{2h})$', '52', '$Pnna$', [['u', '1/2', 'w']]],
 ['Orthorhombic $(D_{2h})$', '53', '$Pmna$', [['u', 'v', '1/2']]],
 ['Orthorhombic $(D_{2h})$', '54', '$Pcca$', [['1/2', 'v', 'w']]],
 ['Orthorhombic $(D_{2h})$',
  '55',
  '$Pbam$',
  [['1/2', 'v', 'w'], ['u', '1/2', 'w']]],
 ['Orthorhombic $(D_{2h})$',
  '56',
  '$Pccn$',
  [['1/2', 'v', 'w'], ['u', '1/2', 'w']]],
 ['Orthorhombic $(D_{2h})$',
  '57',
  '$Pbcm$',
  [['u', 'v', '1/2'], ['u', '1/2', 'w']]],
 ['Orthorhombic $(D_{2h})$',
  '58',
  '$Pnnm$',
  [['1/2', 'v', 'w'], ['u', '1/2', 'w']]],
 ['Orthorhombic $(D_{2h})$',
  '59',
  '$Pmmn$',
  [['1/2', 'v', 'w'], ['u', '1/2', 'w']]],
 ['Orthorhombic $(D_{2h})$',
  '60',
  '$Pbcn$',
  [['1/2', 'v', 'w'], ['u', 'v', '1/2']]],
 ['Orthorhombic $(D_{2h})$',
  '61',
  '$Pbc

In [509]:
kdict = {}
def isinplane(v,plane):
    for i in range(3):
        if(plane[i]=="1/2" and v[i]!="1/2"):
            return False
        if(plane[i]=="0" and v[i]!="0"):
            return False
    return True
    
for el in table1_arr:
    (pname , pnum, pstr, planelist) = el
    print(pnum, pname, pstr, planelist)
    kdict[pnum]={}
    momentumdict = get_momenta_dict(int(pnum))
    for (k,v) in momentumdict.items():
        
        isin = False
        plainlist=[]
        for plane in planelist:
            if(isinplane(v,plane)):
                isin = True
                plainlist.append(plane)

        if(isin):
            # 63, 64(T, Y) have 1/2,0,1 anyways not on the plane
            # 176, 193, 194(H) have (1/3,1/3,1/2)
            # 64 R exception but why?
            if not (v[0] in ['1/2','0','1'] and v[1] in ['1/2','0','1'] and v[2] in ['1/2','0','1']):
                print(pnum,":",k," is not in the first Brillouin zone")

            kdict[pnum][k]={
                "kvec":v,
                "planelist":plainlist,
            }

kdict

11 Monoclinic $(C_{2h})$   $P2_1/m$ [['u', '1/2', 'w']]
14 Monoclinic $(C_{2h})$ $P2_1/c$ [['u', '1/2', 'w']]
51 Orthorhombic $(D_{2h})$ $Pmma$ [['1/2', 'v', 'w']]
52 Orthorhombic $(D_{2h})$ $Pnna$ [['u', '1/2', 'w']]
53 Orthorhombic $(D_{2h})$ $Pmna$ [['u', 'v', '1/2']]
54 Orthorhombic $(D_{2h})$ $Pcca$ [['1/2', 'v', 'w']]
55 Orthorhombic $(D_{2h})$ $Pbam$ [['1/2', 'v', 'w'], ['u', '1/2', 'w']]
56 Orthorhombic $(D_{2h})$ $Pccn$ [['1/2', 'v', 'w'], ['u', '1/2', 'w']]
57 Orthorhombic $(D_{2h})$ $Pbcm$ [['u', 'v', '1/2'], ['u', '1/2', 'w']]
58 Orthorhombic $(D_{2h})$ $Pnnm$ [['1/2', 'v', 'w'], ['u', '1/2', 'w']]
59 Orthorhombic $(D_{2h})$ $Pmmn$ [['1/2', 'v', 'w'], ['u', '1/2', 'w']]
60 Orthorhombic $(D_{2h})$ $Pbcn$ [['1/2', 'v', 'w'], ['u', 'v', '1/2']]
61 Orthorhombic $(D_{2h})$ $Pbca$ [['1/2', 'v', 'w'], ['u', 'v', '1/2'], ['u', '1/2', 'w']]
62 Orthorhombic $(D_{2h})$ $Pnma$ [['1/2', 'v', 'w'], ['u', 'v', '1/2'], ['u', '1/2', 'w']]
63 Orthorhombic $(D_{2h})$ $Cmcm$ [['u', 'v', '1/2']

{'11': {'C': {'kvec': ['1/2', '1/2', '0'], 'planelist': [['u', '1/2', 'w']]},
  'D': {'kvec': ['0', '1/2', '1/2'], 'planelist': [['u', '1/2', 'w']]},
  'E': {'kvec': ['1/2', '1/2', '1/2'], 'planelist': [['u', '1/2', 'w']]},
  'Z': {'kvec': ['0', '1/2', '0'], 'planelist': [['u', '1/2', 'w']]}},
 '14': {'C': {'kvec': ['1/2', '1/2', '0'], 'planelist': [['u', '1/2', 'w']]},
  'D': {'kvec': ['0', '1/2', '1/2'], 'planelist': [['u', '1/2', 'w']]},
  'E': {'kvec': ['1/2', '1/2', '1/2'], 'planelist': [['u', '1/2', 'w']]},
  'Z': {'kvec': ['0', '1/2', '0'], 'planelist': [['u', '1/2', 'w']]}},
 '51': {'R': {'kvec': ['1/2', '1/2', '1/2'], 'planelist': [['1/2', 'v', 'w']]},
  'S': {'kvec': ['1/2', '1/2', '0'], 'planelist': [['1/2', 'v', 'w']]},
  'U': {'kvec': ['1/2', '0', '1/2'], 'planelist': [['1/2', 'v', 'w']]},
  'X': {'kvec': ['1/2', '0', '0'], 'planelist': [['1/2', 'v', 'w']]}},
 '52': {'R': {'kvec': ['1/2', '1/2', '1/2'], 'planelist': [['u', '1/2', 'w']]},
  'S': {'kvec': ['1/2', '1/2', '0']

In [496]:
for seq in eightfold:
    (pnum,k,rep) = seq.split(":")
    if not(k in kdict[pnum]):
        print(pnum,k,rep)

In [497]:
exceptionPoints={"176:H", "193:H", "194:H", "64:"}
get_momenta_dict(176)

{'L': ['1/2', '0', '1/2'],
 'M': ['1/2', '0', '0'],
 'H': ['1/3', '1/3', '1/2'],
 'K': ['1/3', '1/3', '0'],
 'A': ['0', '0', '1/2'],
 'GM': ['0', '0', '0']}

In [470]:
sigmas=[
    "54(U1U2),54(R1R2),56(U1U2),60(R1R2),61(S1S2),62(S1S2),205(M1M2)",
    "52(S1S2),56(T1T2),57(T1T2),57(R1R2),61(T1T2),130(R1R2),138(R1R2)",
    "60(T1T2),60(U1U2),61(U1U2),62(R1R2),128(A3A4),137(A3A4) ,176(A2A3),193(A3),194(A3)"
]
for el1 in sigmas:
    print(el1)
    for el in el1.split(','):
        [a,b]=el.split('(')
        key=a+":"+b[0]+":"+b.split(')')[0]
        print(eightfold[key])

54(U1U2),54(R1R2),56(U1U2),60(R1R2),61(S1S2),62(S1S2),205(M1M2)
(D_{2h:xyz})Ag + 2B1g + 2B2g + B3g + 2Au + B1u + B2u + [Ag] + [B3g] + [B1u] + [B2u] + 2[B3u]
(D_{2h:xyz})Ag + 2B1g + 2B2g + B3g + 2Au + B1u + B2u + [Ag] + [B3g] + [B1u] + [B2u] + 2[B3u]
(D_{2h:xyz})Ag + 2B1g + 2B2g + B3g + 2Au + B1u + B2u + [Ag] + [B3g] + [B1u] + [B2u] + 2[B3u]
(D_{2h:xyz})Ag + 2B1g + 2B2g + B3g + Au + 2B2u + B3u + [Ag] + [B3g] + [Au] + 2[B1u] + [B3u]
(D_{2h:xyz})Ag + 2B1g + 2B2g + B3g + Au + 2B1u + B3u + [Ag] + [B3g] + [Au] + 2[B2u] + [B3u]
(D_{2h:xyz})Ag + 2B1g + 2B2g + B3g + Au + 2B1u + B3u + [Ag] + [B3g] + [Au] + 2[B2u] + [B3u]
(D_{2h:xyz})Ag + 2B1g + 2B2g + B3g + Au + 2B1u + B3u + [Ag] + [B3g] + [Au] + 2[B2u] + [B3u]
52(S1S2),56(T1T2),57(T1T2),57(R1R2),61(T1T2),130(R1R2),138(R1R2)
(D_{2h:xyz})Ag + 2B1g + B2g + 2B3g + 2Au + B1u + B3u + [Ag] + [B2g] + [B1u] + 2[B2u] + [B3u]
(D_{2h:xyz})Ag + 2B1g + B2g + 2B3g + 2Au + B1u + B3u + [Ag] + [B2g] + [B1u] + 2[B2u] + [B3u]
(D_{2h:xyz})Ag + 2B1g + B2g + 2B3g + A

In [467]:
for key in eightfold:
    print(key,eightfold[key])


52:S:S1S2 (D_{2h:xyz})Ag + 2B1g + B2g + 2B3g + 2Au + B1u + B3u + [Ag] + [B2g] + [B1u] + 2[B2u] + [B3u]
60:T:T1T2 (D_{2h:xyz})Ag + B1g + 2B2g + 2B3g + 2Au + B2u + B3u + [Ag] + [B1g] + 2[B1u] + [B2u] + [B3u]
60:U:U1U2 (D_{2h:xyz})Ag + B1g + 2B2g + 2B3g + Au + B1u + 2B2u + [Ag] + [B1g] + [Au] + [B1u] + 2[B3u]
60:R:R1R2 (D_{2h:xyz})Ag + 2B1g + 2B2g + B3g + Au + 2B2u + B3u + [Ag] + [B3g] + [Au] + 2[B1u] + [B3u]
62:S:S1S2 (D_{2h:xyz})Ag + 2B1g + 2B2g + B3g + Au + 2B1u + B3u + [Ag] + [B3g] + [Au] + 2[B2u] + [B3u]
62:R:R1R2 (D_{2h:xyz})Ag + B1g + 2B2g + 2B3g + 2B1u + B2u + B3u + [Ag] + [B1g] + 2[Au] + [B2u] + [B3u]
128:A:A3A4 (D_{4h})A1g + A2g + 2B1g + 2B2g + A1u + A2u + 2B2u + [A1g] + [A2g] + [A1u] + [A2u] + 2[B1u]
205:M:M1M2 (D_{2h:xyz})Ag + 2B1g + 2B2g + B3g + Au + 2B1u + B3u + [Ag] + [B3g] + [Au] + 2[B2u] + [B3u]
54:U:U1U2 (D_{2h:xyz})Ag + 2B1g + 2B2g + B3g + 2Au + B1u + B2u + [Ag] + [B3g] + [B1u] + [B2u] + 2[B3u]
54:R:R1R2 (D_{2h:xyz})Ag + 2B1g + 2B2g + B3g + 2Au + B1u + B2u + [Ag] + [B3g

In [64]:
sgtotype = {}
fname="latexchunks/table2.tex"
classname = ""
with open(fname) as f:
    lines = f.readlines()
    reptxt = ""
    for line in lines:
        line=line.strip()
        if(len(line)==0):
            continue
        if(line[0]=="$"):
            #Class line
            classname = line.split("&")[0].strip()
            #print(classname)
            continue
        line = line.split("\\multicolumn{2}{|c|}{")
        if(len(line)<2):
            continue
        line = line[1][:-3]
        print(line)
        if(line[:4]=="\\(H=" or line[:4]=="\\(~+" or line[:3]=="\\(+"):
            continue
        cols = line.split("\\), \\(")
        print(cols)
        for col in cols:
            col = col.replace("\\(","").replace("\\)","")
            col = col.split("(")
            for el in col:
                sgtotype[el]=classname
        #print(line)

\(H=\epsilon_0+ (t_{1x}k_x+t_{1z}k_z)k_y\tau_1+ t_2k_y\tau_2\)
\(~+\tau_3[\lambda_xk_y\sigma_x+(\lambda_{yx}k_x+\lambda_{yz}k_z)\sigma_y+\lambda_zk_y\sigma_z]\)}\\ \hl
\(11(C_1, D_1, E_1, Z_1)\),~\(14(Z_1)\),
['\\(11(C_1, D_1, E_1, Z_1)\\),~\\(14(Z_1)\\),']
\(63(R_1(yz))\), \(176(L_1(yz))\)}\\\hline \hl
['\\(63(R_1(yz))', '176(L_1(yz))\\)}\\\\\\hline \\hl']
\(H=\epsilon_0+(t_{1x}k_x+t_{1z}k_z)k_y \tau_1+(t_{3x}k_x+t_{3z}k_z)k_y\tau_3\)
\(~+\tau_2[(\lambda_{xx}k_x +\lambda_{xz}k_z)k_y\sigma_x+\lambda_y\sigma_y+(\lambda_{zx}k_x+\lambda_{zz}k_z)k_y\sigma_z]\)}\\ \hl
\(14(D_1^\pm D_2^\pm)\), \(64(R_1^\pm R_2^\pm(yz))\)}\\ \hline \hl
['\\(14(D_1^\\pm D_2^\\pm)', '64(R_1^\\pm R_2^\\pm(yz))\\)}\\\\ \\hline \\hl']
\(H=\epsilon_0+ (t_{1x}k_x+t_{1y}k_y)k_z\tau_1+ (t_{2x}k_x+t_{2y}k_y)\tau_2\)
\(+\tau_3[(\lambda_{xx}k_x+\lambda_{xy}k_y)\sigma_x+(\lambda_{yx}k_x+\lambda_{yy}k_y)\sigma_y+\lambda_z k_z\sigma_z]\)}\\ \hl
\(64(S_1)\)}\\ \hline \hl
['\\(64(S_1)\\)}\\\\ \\hline \\hl']
\(H=\epsilon_0+ t_

In [57]:
("abd").split("a")

['', 'bd']

In [ ]:
with open("magnetite_test/data001.qlw", "r", encoding=""

In [2]:
data

NameError: name 'data' is not defined

In [3]:
invMap

NameError: name 'invMap' is not defined